In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy import stats
from tableone import TableOne
from statannotations.Annotator import Annotator
import numpy as np
from scipy.stats import fisher_exact
import itertools
import statannotations
import scipy
import matplotlib as mpl
import sys

# Configure matplotlib and seaborn for consistent plotting style
plt.rcParams['font.family'] = ['Arial', 'DejaVu Sans', 'sans-serif']
sns.set_theme(style="ticks")
sns.set_context("notebook")
sns.set_palette("Pastel1")

pastel_rainbow = [
    "#F0C2C2",  # soft coral pink
    "#F0D2B8",  # soft peach
    "#FFFFBA",  # soft butter yellow
    "#C2F0C8",  # soft mint green
    "#C2D8F0",  # soft sky blue
    "#D8C2F0"   # soft lilac
]

# Show 100 rows in pandas dataframes by default
pd.set_option('display.max_rows', 100)

# Print Python and package versions for reproducibility
print(f'Python: {sys.version}')

packages = [
    ('pandas', 'pd'),
    ('matplotlib', 'matplotlib'), 
    ('seaborn', 'sns'),
    ('scipy', 'scipy'),
    ('tableone', 'tableone'),
    ('statannotations', 'statannotations'),
    ('numpy', 'np')
]

for package_name, import_name in packages:
    try:
        module = __import__(package_name)
        print(f'{package_name}: {module.__version__}')
    except Exception as e:
        print(f'{package_name}: Error - {e}')

# Data Loading and Initial Preprocessing

In [ ]:
df = pd.read_csv('ExaminingExperiences_DATA_LABELS_2025-07-09_0751_deid_cleaned.csv', index_col=0)
df.shape

In [ ]:
# Simplify 'multiple_interviews' column
df['multiple_interviews'] = (
    ((df['Did you interview at and receive offers from multiple institutions?'].fillna('') == 'Yes') |
     (df['Did you interview at and receive offers from multiple institutions?.1'].fillna('') == 'Yes'))
    &
    (df['Did you interview at and receive offers from multiple institutions?'].fillna('') != 'None') &
    (df['Did you interview at and receive offers from multiple institutions?.1'].fillna('') != 'None')
)
df.groupby('Type of Job Most Recently Accepted')['multiple_interviews'].value_counts()

In [ ]:
# Create 'extra_degree' column
df['extra_degree'] = (
    (df['Degrees Held (Select all that apply) (choice=MS (including MSCE, MSCI, MSci, MPH, etc.))'] == 'Checked') |
    (df['Degrees Held (Select all that apply) (choice=PhD)'] == 'Checked')
)
df['extra_degree'].value_counts()

In [ ]:
# Clean 'gender' column
df['gender'] = df['Gender'].replace('Prefer not to answer', np.nan)
df.gender.value_counts()

In [ ]:
# Replace 'Unchecked' with NaN and create a main copy
df = df.replace('Unchecked', np.nan)
df_main = df.copy()

# TableOne - Demographics

In [ ]:
# Demographics TableOne by respondent
columns = [
 'Gender',
  'Degrees Held (Select all that apply) (choice=MD, DO, MBBS or equivalent)',
 'Degrees Held (Select all that apply) (choice=MS (including MSCE, MSCI, MSci, MPH, etc.))',
 'Degrees Held (Select all that apply) (choice=PhD)',
  'Race/Ethnicity (Select all that apply) (choice=Asian or Asian American)',
 'Race/Ethnicity (Select all that apply) (choice=Black or African American)',
 'Race/Ethnicity (Select all that apply) (choice=Hispanic, Latino, Latina, or LatinX)',
 'Race/Ethnicity (Select all that apply) (choice=White)',
 'Race/Ethnicity (Select all that apply) (choice=Other)',
 'Race/Ethnicity (Select all that apply) (choice=Prefer not to answer)',
  'Do you identify as underrepresented in medicine?',
 'Did you accept a job at the same institution where you completed fellowship training?',
 'multiple_interviews',
   'Type of Research (select all that apply) (choice=Basic)',
 'Type of Research (select all that apply) (choice=Translational)',
 'Type of Research (select all that apply) (choice=Clinical)',
 'Type of Research (select all that apply) (choice=Computational (only dry lab))',
 'Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?',
]

table1 = TableOne(df, columns=columns, categorical=columns, missing=False,
                  rename=None, groupby='Type of Job Most Recently Accepted', pval=True)
table1.to_csv('table1_demographics.csv')
table1

# Data Splitting and Cleaning for Physician Scientist (PS) and Clinician Educator (CE) Cohorts

In [ ]:
# Split by PS CE
df_scientist = df_main[df_main['Type of Job Most Recently Accepted'] == 'Physician scientist'].copy()
df_educator = df_main[df_main['Type of Job Most Recently Accepted'] == 'Clinician educator'].copy()

# Drop columns where all values are NaN or empty strings for df_scientist
df_scientist = df_scientist.dropna(axis=1, how='all')         # Remove columns that are all NaN
df_scientist = df_scientist.loc[:, ~(df_scientist == '').all()]  # Remove columns that are all empty strings
df_scientist.shape

In [ ]:
# Columns to drop from df_scientist
columns_to_drop = [
    "Do you agree to participate in this survey? By clicking 'I Agree', you are consenting to participate in this survey. ",
    'Degrees Held (Select all that apply) (choice=MD, DO, MBBS or equivalent)',
    'Degrees Held (Select all that apply) (choice=MS (including MSCE, MSCI, MSci, MPH, etc.))',
    'Degrees Held (Select all that apply) (choice=PhD)',
    'Degrees Held (Select all that apply) (choice=Other)',
    'If Other for Degrees Held, please specify:',
    'Gender',
    'Race/Ethnicity (Select all that apply) (choice=Asian or Asian American)',
    'Race/Ethnicity (Select all that apply) (choice=Hispanic, Latino, Latina, or LatinX)',
    'Race/Ethnicity (Select all that apply) (choice=White)',
    'Race/Ethnicity (Select all that apply) (choice=Prefer not to answer)'
]

# Drop the columns (only drop columns that actually exist in the dataframe)
columns_to_drop_existing = [col for col in columns_to_drop if col in df_scientist.columns]
df_scientist = df_scientist.drop(columns=columns_to_drop_existing)

print(f"Dropped {len(columns_to_drop_existing)} columns from df_scientist")
print(f"Remaining columns: {df_scientist.shape[1]}")

In [ ]:
# Drop columns where all values are NaN or empty strings for df_educator
df_educator = df_educator.dropna(axis=1, how='all')         # Remove columns that are all NaN
df_educator = df_educator.loc[:, ~(df_educator == '').all()]  # Remove columns that are all empty strings
df_educator.shape

In [ ]:
# Columns to drop from df_educator
columns_to_drop = [
    "Do you agree to participate in this survey? By clicking 'I Agree', you are consenting to participate in this survey. ",
    'Degrees Held (Select all that apply) (choice=MD, DO, MBBS or equivalent)',
    'Degrees Held (Select all that apply) (choice=MS (including MSCE, MSCI, MSci, MPH, etc.))',
    'Degrees Held (Select all that apply) (choice=PhD)',
    'Degrees Held (Select all that apply) (choice=Other)',
    'If Other for Degrees Held, please specify:',
    'Gender',
    'Race/Ethnicity (Select all that apply) (choice=Asian or Asian American)',
    'Race/Ethnicity (Select all that apply) (choice=Hispanic, Latino, Latina, or LatinX)',
    'Race/Ethnicity (Select all that apply) (choice=White)',
    'Race/Ethnicity (Select all that apply) (choice=Prefer not to answer)'
]

# Drop the columns (only drop columns that actually exist in the dataframe)
columns_to_drop_existing = [col for col in columns_to_drop if col in df_educator.columns]
df_educator = df_educator.drop(columns=columns_to_drop_existing)

print(f"Dropped {len(columns_to_drop_existing)} columns from df_educator")
print(f"Remaining columns: {df_educator.shape[1]}")

In [ ]:
# Re-check shape after dropping columns (redundant, but kept for consistency with original notebook)
df_educator = df_educator.dropna(axis=1, how='all')         # Remove columns that are all NaN
df_educator = df_educator.loc[:, ~(df_educator == '').all()]  # Remove columns that are all empty strings
df_educator.shape

# PS Job Offer Melting and Analysis

In [ ]:
df_scientist.shape

In [ ]:
def melt_job_offers_ps(df_scientist):
    """
    Melt job offer columns from wide to long format for Physician Scientists.
    Each respondent can have up to 3 job offers (base, .1, .2 suffixes).
    """
    
    # Define the base column names (without suffixes)
    base_columns = [
        'What salary range were you offered?',
        'What were you offered for protected research time?',
        'What geographic region is the institution?',
        'Cost of living at institution (example cities)',
        'Did you receive or negotiate a startup package as part of your appointment?',
        'If yes, what is the approximate value of your startup package?',
        'Did the startup package have salary cost-sharing? (e.g., your grant does not cover all your salary and hence you need to pay the differential via your startup funds)',
        'Were any signing bonuses or performance-based incentives part of your negotiation?',
        'If yes, what is the approximate total value of these bonuses/incentives?',
        'Were you offered a relocation package?',
        'If yes, what was the total value of the relocation package?',
        'Do you have a noncompete clause?',
    ]
    
    # Get all non-job-offer columns (these will be preserved for each respondent)
    job_offer_columns = []
    for base_col in base_columns:
        job_offer_columns.extend([base_col, base_col + '.1', base_col + '.2'])
    
    # Identify columns that are NOT job offer columns
    id_columns = [col for col in df_scientist.columns if col not in job_offer_columns]
    
    # Create separate dataframes for each job offer
    dfs_to_concat = []
    
    for offer_num, suffix in enumerate(['', '.1', '.2']):
        # Get columns for this job offer
        offer_columns = {}
        for base_col in base_columns:
            full_col = base_col + suffix
            if full_col in df_scientist.columns:
                offer_columns[full_col] = base_col
        
        if offer_columns:  # Only process if columns exist
            # Select ID columns + this offer's columns
            cols_to_select = id_columns + list(offer_columns.keys())
            df_offer = df_scientist[cols_to_select].copy()
            
            # Rename columns to remove suffixes
            df_offer = df_offer.rename(columns=offer_columns)
            
            # Add job offer number
            df_offer['job_offer_number'] = offer_num + 1
            
            # Remove rows where all job offer columns are null
            job_cols_mask = df_offer[base_columns].notna().any(axis=1)
            df_offer = df_offer[job_cols_mask]
            
            dfs_to_concat.append(df_offer)
    
    # Concatenate all job offers
    df_melted = pd.concat(dfs_to_concat, ignore_index=True)
    
    # Sort by respondent identifier and job offer number
    # If you have a respondent ID column, replace 'index' with that column name
    if 'respondent_id' in df_melted.columns:
        df_melted = df_melted.sort_values(['respondent_id', 'job_offer_number'])
    else:
        # If no explicit ID, sort by index and job offer number
        df_melted = df_melted.sort_values(['job_offer_number'])
    
    return df_melted

# Example usage:
df_scientist_melted = melt_job_offers_ps(df_scientist)
print(f"Original shape: {df_scientist.shape}")
print(f"Melted shape: {df_scientist_melted.shape}")
print(f"Number of unique job offers: {len(df_scientist_melted)}")
df_scientist_melted.job_offer_number.value_counts()

In [ ]:
df_scientist_melted.to_csv('df_scientist_melted.csv', index=False)

In [ ]:
# Free text columns to drop from df_scientist_melted
freetext_columns_to_drop = [
    "Do you agree to participate in this survey? By clicking 'I Agree', you are consenting to participate in this survey.  ",
    'If yes, please provide details regarding radius, buyout, and time frame.',
    'Why did you choose to accept this job over other offers?',
    'Would you like to add any additional comments about this first job offer?',
    'If yes, please provide details regarding radius, buyout, and time frame..1',
    'Would you like to add any additional comments about this second job offer?',
    'Did you interview at and receive offers from yet another institution?',
    'If yes, please provide details regarding radius, buyout, and time frame..2',
    'Did you interview at and receive offers from yet another institution? (You will not be asked to complete another set of questions.)',
    'Complete?'
]

# Drop the columns (only drop columns that actually exist in the dataframe)
freetext_columns_existing = [col for col in freetext_columns_to_drop if col in df_scientist_melted.columns]
df_scientist_melted = df_scientist_melted.drop(columns=freetext_columns_existing)

print(f"Dropped {len(freetext_columns_existing)} free text columns from df_scientist_melted")
print(f"Remaining columns: {df_scientist_melted.shape[1]}")
print(f"Remaining rows: {df_scientist_melted.shape[0]}")

# TableOne Generation - PS Job Offers

In [ ]:
columns = [     'What salary range were you offered?',
       'What were you offered for protected research time?',
       'What geographic region is the institution?',
       'Cost of living at institution (example cities)',
       'Did you receive or negotiate a startup package as part of your appointment?',
       'If yes, what is the approximate value of your startup package?',
       'Did the startup package have salary cost-sharing? (e.g., your grant does not cover all your salary and hence you need to pay the differential via your startup funds)',
       'Were any signing bonuses or performance-based incentives part of your negotiation?',
       'If yes, what is the approximate total value of these bonuses/incentives?',
       'Were you offered a relocation package?',
       'If yes, what was the total value of the relocation package?',
       'Do you have a noncompete clause?', 'job_offer_number']



# Generate TableOne
table1 = TableOne(df_scientist_melted, columns=columns, categorical=columns,  rename=None, groupby=
 'Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?', pval=True)
table1.to_csv('table1_job_offers_scientist_byK.csv')
table1

# PS Job Offer Plotting Functions

In [ ]:
# Fix stayed at same institution flag
df_scientist_melted['Did you accept a job at the same institution where you completed fellowship training?'].value_counts()

In [ ]:
df_scientist_melted['internal_offer'] = ((df_scientist_melted['Did you accept a job at the same institution where you completed fellowship training?'] == 'Yes') & 
                              (df_scientist_melted['job_offer_number'] == 1)).apply(lambda x: True if x else False)

In [ ]:
df_scientist_melted['internal_offer'].value_counts()

In [ ]:
# Define the ordered categories for salary
salary_categories = [
    'Less than $150,000',
    '$150,000-$199,999',
    '$200,000-$249,999',
    '$250,000-$299,999',
    'More than $300,000'
]

# Convert to categorical with ordering
df_scientist_melted['salary_range_offered'] = pd.Categorical(df_scientist_melted['What salary range were you offered?'], categories=salary_categories, ordered=True)

salary_map = {
    'Less than $150,000': 1,
    '$150,000-$199,999': 2,
    '$200,000-$249,999': 3,
    '$250,000-$299,999': 4,
    'More than $300,000': 5
}

df_scientist_melted['salary_ordinal'] = df_scientist_melted['salary_range_offered'].map(salary_map)

## PS Salary Plots

In [ ]:
def create_stacked_bar_with_stats(data, x_col, y_col, order=None, pairs=None, 
                                 test='Mann-Whitney', loc='outside', 
                                 text_format='simple', figsize=(7, 4), title=None, xlabel=None):
    
    # Create salary mapping
    salary_map = {
        'Less than $150,000': 1,
        '$150,000-$199,999': 2,
        '$200,000-$249,999': 3,
        '$250,000-$299,999': 4,
        'More than $300,000': 5
    }
    
    # Create reverse mapping for labels
    reverse_salary_map = {v: k for k, v in salary_map.items()}
    
    # Prepare data for stacked bar chart
    df_plot = data.copy()
    df_plot['salary_range'] = df_plot[y_col].map(reverse_salary_map)
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[x_col], df_plot['salary_range'])
    
    # Reorder columns based on salary_ordinal order
    salary_order = [reverse_salary_map[i] for i in sorted(reverse_salary_map.keys())]
    crosstab = crosstab.reindex(columns=salary_order, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)

    # Create the stacked bar plot
    fig, ax = plt.subplots(figsize=figsize)
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax,
                       color=pastel_rainbow)
    
    # Customize the plot
    ax.set_xlabel(xlabel or x_col)
    ax.set_ylabel('Proportion')
    ax.set_title(title or f'Distribution of Salary Ranges by {x_col}')
    ax.set_ylim(0, 1)  # Set y-axis from 0 to 1 for proportions
    
    # Fix legend order to match stacking order (flip it)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Salary Range', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.tick_params(axis='x', rotation=15)
    
    # Add statistical annotation if pairs are provided
    if pairs:
        annotator = Annotator(ax, pairs, data=data, x=x_col, y=y_col, order=order)
        annotator.configure(test=test, loc=loc, text_format=text_format)
        annotator.apply_and_annotate()
    
    plt.tight_layout()
    return fig, ax

# Create plot for K08/K23 grant status vs. salary
fig, ax = create_stacked_bar_with_stats(
    data=df_scientist_melted, 
    x_col='Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?',
    y_col='salary_ordinal', 
    order=['No','Yes'],
    pairs=[('No','Yes')],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges',
    xlabel='K08/K23 already?'
)
plt.show()

In [ ]:
research_type_columns = [
    'Type of Research (select all that apply) (choice=Basic)',
    'Type of Research (select all that apply) (choice=Translational)', 
    'Type of Research (select all that apply) (choice=Clinical)',
    'Type of Research (select all that apply) (choice=Computational (only dry lab))'
]

# Shorter labels for plotting
research_labels = {
    'Type of Research (select all that apply) (choice=Basic)': 'Basic\nResearch',
    'Type of Research (select all that apply) (choice=Translational)': 'Translational\nResearch',
    'Type of Research (select all that apply) (choice=Clinical)': 'Clinical\nResearch',
    'Type of Research (select all that apply) (choice=Computational (only dry lab))': 'Computational\nResearch'
}

# Define salary categories in order
salary_categories = [
    'Less than $150,000',
    '$150,000-$199,999',
    '$200,000-$249,999',
    '$250,000-$299,999',
    'More than $300,000'
]

# Create salary mapping
salary_map = {
    'Less than $150,000': 1,
    '$150,000-$199,999': 2,
    '$200,000-$249,999': 3,
    '$250,000-$299,999': 4,
    'More than $300,000': 5
}

# Create reverse mapping for labels
reverse_salary_map = {v: k for k, v in salary_map.items()}

def create_combined_research_salary_plot(data, salary_col):
    """Create a single plot with stacked bars for each research type showing salary distribution"""
    
    # Check which columns actually exist
    existing_research_cols = [col for col in research_type_columns if col in data.columns]
    print(f"Found {len(existing_research_cols)} research type columns in data")
    
    if len(existing_research_cols) == 0:
        print("No research type columns found!")
        return None, None
    
    # Prepare data for all research types
    all_proportions = []
    research_names = []
    sample_sizes = []
    
    for research_col in existing_research_cols:
        # Filter data for this research type where value is 'Checked'
        try:
            research_mask = data[research_col] == 'Checked'
            research_data = data[research_mask].copy()
        except Exception as e:
            print(f"Error filtering {research_col}: {e}")
            continue
        
        if len(research_data) == 0:
            print(f"No data for {research_col}")
            continue
        
        # Create value counts for salary ranges
        # If using ordinal values, convert back to text labels
        if salary_col == 'salary_ordinal':
            # Map ordinal values back to text labels
            research_data_mapped = research_data.copy()
            research_data_mapped['salary_text'] = research_data_mapped[salary_col].map(reverse_salary_map)
            salary_counts = research_data_mapped['salary_text'].value_counts()
        else:
            salary_counts = research_data[salary_col].value_counts()
        
        # Reorder based on salary categories
        salary_counts = salary_counts.reindex(salary_categories, fill_value=0)
        
        # Convert to proportions
        salary_proportions = salary_counts / salary_counts.sum()
        
        all_proportions.append(salary_proportions.values)
        research_names.append(research_labels.get(research_col, research_col.split('=')[1].replace(')', '')))
        sample_sizes.append(len(research_data))
    
    if len(all_proportions) == 0:
        print("No valid data found for any research type!")
        return None, None
    
    # Convert to numpy array for easier plotting
    proportions_array = np.array(all_proportions).T  # Transpose so categories are rows, research types are columns
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(7, 4))

    
    # Create stacked bars
    bottom = np.zeros(len(research_names))
    
    for i, category in enumerate(salary_categories):
        if i < len(proportions_array):
            bars = ax.bar(research_names, proportions_array[i], 
                         bottom=bottom, label=category, color=pastel_rainbow[i])
            bottom += proportions_array[i]
    
    # Customize the plot
    ax.set_ylabel('Proportion', fontsize=12)
    ax.set_xlabel('Research Type', fontsize=12)
    ax.set_title('Salary Distribution by Research Type', fontsize=12, )
    ax.set_ylim(0, 1)
    
    # Add sample sizes to x-axis labels
    x_labels_with_n = [f"{name}\n(n={n})" for name, n in zip(research_names, sample_sizes)]
    ax.set_xticklabels(x_labels_with_n)
    
    # Create legend (reversed to match stacking order from lowest to highest salary)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Salary Range', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    return fig, ax

In [ ]:
# Create the combined plot for research type vs. salary
fig, ax = create_combined_research_salary_plot(
    data=df_scientist_melted,
    salary_col='salary_ordinal'
)

if fig is not None:
    plt.show()

## PS Salary Plots - Pairwise Comparisons

In [ ]:
# Create plot for multiple interviews vs. salary
fig, ax = create_stacked_bar_with_stats(
    data=df_scientist_melted, 
    x_col='multiple_interviews',
    y_col='salary_ordinal', 
    order=[True,False],
    pairs=[(True,False)],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges'
)
plt.show()

In [ ]:
# Create plot for extra degree vs. salary
fig, ax = create_stacked_bar_with_stats(
    data=df_scientist_melted, 
    x_col='extra_degree',
    y_col='salary_ordinal', 
    order=[True,False],
    pairs=[(True,False)],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges'
)
plt.show()

In [ ]:
# Create plot for gender vs. salary
fig, ax = create_stacked_bar_with_stats(
    data=df_scientist_melted, 
    x_col='gender',
    y_col='salary_ordinal', 
    order=['Man','Woman'],
    pairs=[('Man','Woman')],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges'
)
plt.show()

In [ ]:
df_scientist_salary = df_scientist_melted[['internal_offer', 'salary_range_offered', 'salary_ordinal',
                                           'multiple_interviews', 'extra_degree', 'gender',
                                           'Do you identify as underrepresented in medicine?',]]

## PS Startup Package Analysis

In [ ]:
df_scientist_melted['If yes, what is the approximate value of your startup package?'].value_counts()

In [ ]:
# Create count plot for startup package values
plt.figure(figsize=(8, 3))
sns.countplot(data=df_scientist_melted, 
              y='If yes, what is the approximate value of your startup package?',
              hue='Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?',
              order=df_scientist_melted['If yes, what is the approximate value of your startup package?'].value_counts().index,
              palette=pastel_rainbow)

plt.title('Count of Startup Package Values by K08/K23 Grant Status')
plt.xlabel('Count')
plt.ylabel('Startup Package Value')
plt.legend(title='K08/K23 Grant Already Awarded?', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Define the ordered categories for startup packages
startup_categories = [
    'Under $250,000',
    '$250,000-$499,999',
    '$500,000-$1,000,000',
    'Over $1,000,000'
]

# Convert to categorical with ordering
df_scientist_melted['startup_range_offered'] = pd.Categorical(
    df_scientist_melted['If yes, what is the approximate value of your startup package?'], 
    categories=startup_categories, 
    ordered=True
)

# Create ordinal mapping for startup packages
startup_map = {
    'Under $250,000': 1,
    '$250,000-$499,999': 2,
    '$500,000-$1,000,000': 3,
    'Over $1,000,000': 4
}

df_scientist_melted['startup_ordinal'] = df_scientist_melted['startup_range_offered'].map(startup_map)

In [ ]:
def create_startup_proportion_plot(data, x_col, startup_col, order=None, pairs=None, 
                                 test='Mann-Whitney', loc='inside', 
                                 text_format='simple', figsize=(7, 4), xlabel=None, xticklabels=None, ax=None):
    
    # Prepare data - drop rows with missing values
    df_plot = data[[x_col, startup_col, 'startup_ordinal']].dropna()
    
    # Create crosstab using the actual categorical column
    crosstab = pd.crosstab(df_plot[x_col], df_plot[startup_col])
    
    # Reorder columns based on startup categories order
    existing_categories = [cat for cat in startup_categories if cat in crosstab.columns]
    crosstab = crosstab.reindex(columns=existing_categories, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    # Get sample sizes for each x category
    sample_sizes = crosstab.sum(axis=1)
    
    # Option 5: Monochromatic Blues
    crest_colors = sns.color_palette("crest", n_colors=len(existing_categories))

    # Create the stacked bar plot with proportions
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig = ax.figure
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=crest_colors[:len(existing_categories)])
    
    # Customize the plot
    ax.set_ylabel('Proportion')
    ax.set_title('Distribution of Startup Package Ranges')
    ax.set_ylim(0, 1)
    
    # Fix legend order to match stacking order (flip it)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Startup Package Range', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.tick_params(axis='x', rotation=0)
    
    # Add sample sizes under x-axis labels
    x_labels_orig = ax.get_xticklabels()
    
    # Get the actual order of categories as they appear in the plot
    actual_categories = crosstab_prop.index.tolist()
    
    for i, (label, category) in enumerate(zip(x_labels_orig, actual_categories)):
        n = sample_sizes[category]  # Get sample size for this specific category
        
        if xticklabels:
            # Find the matching custom label for this category
            if order:
                # If order is specified, use the position in the order list
                try:
                    category_index = order.index(category)
                    current_text = xticklabels[category_index] if category_index < len(xticklabels) else category
                except ValueError:
                    current_text = category
            else:
                # If no order specified, use position in the plot
                current_text = xticklabels[i] if i < len(xticklabels) else category
        else:
            # Use original category name
            current_text = category
            
        new_text = f"{current_text}\n(n={n})"
        label.set_text(new_text)
    
    # Set the updated labels
    ax.set_xticklabels(x_labels_orig)
    
    # Set x-axis label if provided
    if xlabel:
        ax.set_xlabel(xlabel)
    
    # Add statistical annotation if pairs are provided
    if pairs:
        annotator = Annotator(ax, pairs, data=df_plot, x=x_col, y='startup_ordinal', order=order)
        annotator.configure(test=test, loc=loc, text_format=text_format)
        annotator.apply_and_annotate()
    
    plt.tight_layout()
    return fig, ax

In [ ]:
# Create plot for K08/K23 grant status vs. startup package
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted,
    x_col='Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?',
    startup_col='If yes, what is the approximate value of your startup package?',
    order=['Yes', 'No'],
    pairs=[('Yes', 'No')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='K08/K23 already?'
)
plt.show()

In [ ]:
df_scientist_melted = df_scientist_melted.rename(columns={
    'Did the startup package have salary cost-sharing? (e.g., your grant does not cover all your salary and hence you need to pay the differential via your startup funds)': 'cost_sharing'
})
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted.loc[df_scientist_melted['cost_sharing'] != 'Not sure'],
    x_col='cost_sharing',
    startup_col='If yes, what is the approximate value of your startup package?',
    order = ['Yes','No'],
    pairs = [('Yes','No')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='Salary cost-sharing in startup package?',
    xticklabels=['Yes', 'No']
)

plt.show()

In [ ]:
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted,
    x_col='gender',
    startup_col='If yes, what is the approximate value of your startup package?',
    order=['Man', 'Woman'],
    pairs=[('Man', 'Woman')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='Gender',
    xticklabels=['Man', 'Woman']
)
plt.show()

In [ ]:
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted,
    x_col='extra_degree',
    startup_col='If yes, what is the approximate value of your startup package?',
    order = [True,False],
    pairs = [(True,False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='Additional Degree?',
    xticklabels=['Yes', 'No']
)

plt.show()

In [ ]:
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted,
    x_col='internal_offer',
    startup_col='If yes, what is the approximate value of your startup package?',
    order = [True,False],
    pairs = [(True,False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='Internal offer?',
    xticklabels=['Yes', 'No']
)

plt.show()

In [ ]:
fig, ax = create_startup_proportion_plot(
    data=df_scientist_melted,
    x_col='multiple_interviews',
    startup_col='If yes, what is the approximate value of your startup package?',
    order = [True,False],
    pairs = [(True,False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    figsize=(7, 4),
    xlabel='Multiple interviews?',
    xticklabels=['Yes', 'No']
)

plt.show()

## PS Figure - Combined Startup Package Plots

In [ ]:
# Create combined figure with 2x2 subplot layout
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
plt.subplots_adjust(hspace=0.4)  # Add more vertical spacing between rows

# Set global font sizes for better readability on printed page
plt.rcParams.update({'font.size': 16})  # Base font size

# Rename cost-sharing column (if not already renamed)
df_scientist_melted = df_scientist_melted.rename(columns={
    'Did the startup package have salary cost-sharing? (e.g., your grant does not cover all your salary and hence you need to pay the differential via your startup funds)': 'cost_sharing'
})

# Define colors once for consistency across all subplots
crest_colors = sns.color_palette("crest", n_colors=len(startup_categories))

# A) K prior to negotiation comparison
ax_a = axes[0, 0]
df_plot_a = df_scientist_melted[['Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?', 
                                'If yes, what is the approximate value of your startup package?', 'startup_ordinal']].dropna()
crosstab_a = pd.crosstab(df_plot_a['Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?'], 
                        df_plot_a['If yes, what is the approximate value of your startup package?'])
existing_categories_a = [cat for cat in startup_categories if cat in crosstab_a.columns]
crosstab_a = crosstab_a.reindex(columns=existing_categories_a, fill_value=0)
crosstab_prop_a = crosstab_a.div(crosstab_a.sum(axis=1), axis=0)
sample_sizes_a = crosstab_a.sum(axis=1)

crosstab_prop_a.plot(kind='bar', stacked=True, ax=ax_a, color=crest_colors[:len(existing_categories_a)], legend=False)
ax_a.set_ylabel('Proportion', fontsize=18)
ax_a.set_title('K08/K23 Prior to Negotiation', fontsize=20)
ax_a.text(-0.12, 1.05, 'A)', transform=ax_a.transAxes, fontsize=24, fontweight='bold')
ax_a.set_ylim(0, 1)
ax_a.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)
ax_a.tick_params(axis='y', labelsize=16)
ax_a.set_xlabel('')  # Set blank x-label

# Add sample sizes and custom labels
x_labels_a = ['Yes', 'No']
actual_categories_a = crosstab_prop_a.index.tolist()
# Map actual categories to custom labels
label_mapping_a = dict(zip(['Yes', 'No'], x_labels_a))
tick_labels_a = [label_mapping_a.get(cat, cat) for cat in actual_categories_a]

for i, category in enumerate(actual_categories_a):
    n = sample_sizes_a[category]
    custom_label = label_mapping_a.get(category, category)
    ax_a.text(i, -0.05, f"{custom_label}\n(n={n})", ha='center', va='top', transform=ax_a.transData, fontsize=15)

ax_a.set_xticklabels(['', ''])  # Clear tick labels since we're using text annotations
ax_a.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)

# Add statistical annotation
annotator_a = Annotator(ax_a, [('Yes', 'No')], data=df_plot_a, 
                       x='Had you already been awarded a K08/K23 or equivalent grant prior to beginning job negotiations?', 
                       y='startup_ordinal', order=['Yes', 'No'])
annotator_a.configure(test='Mann-Whitney', loc='inside', text_format='simple')
annotator_a.apply_and_annotate()

# B) Cost-sharing
ax_b = axes[0, 1]
df_plot_b = df_scientist_melted.loc[df_scientist_melted['cost_sharing'] != 'Not sure']
df_plot_b = df_plot_b[['cost_sharing', 'If yes, what is the approximate value of your startup package?', 'startup_ordinal']].dropna()
crosstab_b = pd.crosstab(df_plot_b['cost_sharing'], df_plot_b['If yes, what is the approximate value of your startup package?'])
existing_categories_b = [cat for cat in startup_categories if cat in crosstab_b.columns]
crosstab_b = crosstab_b.reindex(columns=existing_categories_b, fill_value=0)
crosstab_prop_b = crosstab_b.div(crosstab_b.sum(axis=1), axis=0)
sample_sizes_b = crosstab_b.sum(axis=1)

crosstab_prop_b.plot(kind='bar', stacked=True, ax=ax_b, color=crest_colors[:len(existing_categories_b)], legend=False)
ax_b.set_ylabel('Proportion', fontsize=18)
ax_b.set_title('Salary Cost-sharing', fontsize=20)
ax_b.text(-0.12, 1.05, 'B)', transform=ax_b.transAxes, fontsize=24, fontweight='bold')
ax_b.set_ylim(0, 1)
ax_b.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)
ax_b.tick_params(axis='y', labelsize=16)
ax_b.set_xlabel('')  # Set blank x-label

# Add sample sizes and custom labels
x_labels_b = ['Yes', 'No']
actual_categories_b = crosstab_prop_b.index.tolist()
# Map actual categories to custom labels
label_mapping_b = dict(zip(['Yes', 'No'], x_labels_b))
tick_labels_b = [label_mapping_b.get(cat, cat) for cat in actual_categories_b]

for i, category in enumerate(actual_categories_b):
    n = sample_sizes_b[category]
    custom_label = label_mapping_b.get(category, category)
    ax_b.text(i, -0.05, f"{custom_label}\n(n={n})", ha='center', va='top', transform=ax_b.transData, fontsize=15)

ax_b.set_xticklabels(['', ''])  # Clear tick labels since we're using text annotations
ax_b.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)

# Add statistical annotation
annotator_b = Annotator(ax_b, [('Yes', 'No')], data=df_plot_b, x='cost_sharing', y='startup_ordinal', order=['Yes', 'No'])
annotator_b.configure(test='Mann-Whitney', loc='inside', text_format='simple')
annotator_b.apply_and_annotate()

# C) Gender
ax_c = axes[1, 0]
df_plot_c = df_scientist_melted[['gender', 'If yes, what is the approximate value of your startup package?', 'startup_ordinal']].dropna()
crosstab_c = pd.crosstab(df_plot_c['gender'], df_plot_c['If yes, what is the approximate value of your startup package?'])
existing_categories_c = [cat for cat in startup_categories if cat in crosstab_c.columns]
crosstab_c = crosstab_c.reindex(columns=existing_categories_c, fill_value=0)
crosstab_prop_c = crosstab_c.div(crosstab_c.sum(axis=1), axis=0)
sample_sizes_c = crosstab_c.sum(axis=1)

crosstab_prop_c.plot(kind='bar', stacked=True, ax=ax_c, color=crest_colors[:len(existing_categories_c)], legend=False)
ax_c.set_ylabel('Proportion', fontsize=18)
ax_c.set_title('Gender Comparison', fontsize=20)
ax_c.text(-0.12, 1.05, 'C)', transform=ax_c.transAxes, fontsize=24, fontweight='bold')
ax_c.set_ylim(0, 1)
ax_c.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)
ax_c.tick_params(axis='y', labelsize=16)
ax_c.set_xlabel('')  # Set blank x-label

# Add sample sizes and custom labels
x_labels_c = ['Man', 'Woman']
actual_categories_c = crosstab_prop_c.index.tolist()
# Map actual categories to custom labels
label_mapping_c = dict(zip(['Man', 'Woman'], x_labels_c))
tick_labels_c = [label_mapping_c.get(cat, cat) for cat in actual_categories_c]

for i, category in enumerate(actual_categories_c):
    n = sample_sizes_c[category]
    custom_label = label_mapping_c.get(category, category)
    ax_c.text(i, -0.05, f"{custom_label}\n(n={n})", ha='center', va='top', transform=ax_c.transData, fontsize=15)

ax_c.set_xticklabels(['', ''])  # Clear tick labels since we're using text annotations
ax_c.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)

# Add statistical annotation
annotator_c = Annotator(ax_c, [('Man', 'Woman')], data=df_plot_c, x='gender', y='startup_ordinal', order=['Man', 'Woman'])
annotator_c.configure(test='Mann-Whitney', loc='inside', text_format='simple')
annotator_c.apply_and_annotate()

# D) Multiple Interviews
ax_d = axes[1, 1]
df_plot_d = df_scientist_melted[['multiple_interviews', 'If yes, what is the approximate value of your startup package?', 'startup_ordinal']].dropna()
crosstab_d = pd.crosstab(df_plot_d['multiple_interviews'], df_plot_d['If yes, what is the approximate value of your startup package?'])
existing_categories_d = [cat for cat in startup_categories if cat in crosstab_d.columns]
crosstab_d = crosstab_d.reindex(columns=existing_categories_d, fill_value=0)
crosstab_prop_d = crosstab_d.div(crosstab_d.sum(axis=1), axis=0)
sample_sizes_d = crosstab_d.sum(axis=1)

crosstab_prop_d.plot(kind='bar', stacked=True, ax=ax_d, color=crest_colors[:len(existing_categories_d)])
ax_d.set_ylabel('Proportion', fontsize=18)
ax_d.set_title('Multiple Interviews', fontsize=20)
ax_d.text(-0.12, 1.05, 'D)', transform=ax_d.transAxes, fontsize=24, fontweight='bold')
ax_d.set_ylim(0, 1)
ax_d.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)
ax_d.tick_params(axis='y', labelsize=16)
ax_d.set_xlabel('')  # Set blank x-label

# Add sample sizes and custom labels
x_labels_d = ['Yes', 'No'] # Assuming True/False maps to Yes/No
actual_categories_d = crosstab_prop_d.index.tolist()
# Map actual categories to custom labels
label_mapping_d = dict(zip([True, False], x_labels_d))
tick_labels_d = [label_mapping_d.get(cat, str(cat)) for cat in actual_categories_d]

for i, category in enumerate(actual_categories_d):
    n = sample_sizes_d[category]
    custom_label = label_mapping_d.get(category, str(category))
    ax_d.text(i, -0.05, f"{custom_label}\n(n={n})", ha='center', va='top', transform=ax_d.transData, fontsize=15)

ax_d.set_xticklabels(['', ''])  # Clear tick labels since we're using text annotations
ax_d.tick_params(axis='x', rotation=0, top=False, bottom=True, labeltop=False, labelbottom=True, labelsize=16)

# Add statistical annotation
annotator_d = Annotator(ax_d, [(True, False)], data=df_plot_d, x='multiple_interviews', y='startup_ordinal', order=[True, False])
annotator_d.configure(test='Mann-Whitney', loc='inside', text_format='simple')
annotator_d.apply_and_annotate()

# Get legend from the last subplot and place it outside the figure
handles, labels = ax_d.get_legend_handles_labels()
fig.legend(reversed(handles), reversed(labels), title='Startup Package Range', 
          bbox_to_anchor=(1.02, 0.5), loc='center left', fontsize=16, title_fontsize=18)

# Remove the legend from subplot D
ax_d.get_legend().remove()

plt.tight_layout()
plt.subplots_adjust(right=0.95)  # Make room for the legend
plt.savefig('fig_startup_package_analysis.pdf', format='pdf', bbox_inches='tight')
plt.show()

## PS Research Type vs. Startup Package

In [ ]:
research_type_columns = [
    'Type of Research (select all that apply) (choice=Basic)',
    'Type of Research (select all that apply) (choice=Translational)', 
    'Type of Research (select all that apply) (choice=Clinical)',
    'Type of Research (select all that apply) (choice=Computational (only dry lab))'
]

# Shorter labels for plotting
research_labels = {
    'Type of Research (select all that apply) (choice=Basic)': 'Basic\nResearch',
    'Type of Research (select all that apply) (choice=Translational)': 'Translational\nResearch',
    'Type of Research (select all that apply) (choice=Clinical)': 'Clinical\nResearch',
    'Type of Research (select all that apply) (choice=Computational (only dry lab))': 'Computational\nResearch'
}

# Define startup package categories in order
startup_categories = [
    'Under $250,000',
    '$250,000-$499,999',
    '$500,000-$1,000,000',
    'Over $1,000,000'
]

def create_combined_research_startup_plot(data, startup_col):
    """Create a single plot with stacked bars for each research type showing startup package distribution"""
    
    # Check which columns actually exist
    existing_research_cols = [col for col in research_type_columns if col in data.columns]
    print(f"Found {len(existing_research_cols)} research type columns in data")
    
    if len(existing_research_cols) == 0:
        print("No research type columns found!")
        return None, None
    
    # Prepare data for all research types
    all_proportions = []
    research_names = []
    sample_sizes = []
    
    for research_col in existing_research_cols:
        # Filter data for this research type where value is 'Checked'
        try:
            research_mask = data[research_col] == 'Checked'
            research_data = data[research_mask].copy()
        except Exception as e:
            print(f"Error filtering {research_col}: {e}")
            continue
        
        if len(research_data) == 0:
            print(f"No data for {research_col}")
            continue
        
        # Create value counts for startup packages
        startup_counts = research_data[startup_col].value_counts()
        
        # Reorder based on startup categories
        existing_categories = [cat for cat in startup_categories if cat in startup_counts.index]
        startup_counts = startup_counts.reindex(startup_categories, fill_value=0)
        
        # Convert to proportions
        startup_proportions = startup_counts / startup_counts.sum()
        
        all_proportions.append(startup_proportions.values)
        research_names.append(research_labels.get(research_col, research_col.split('=')[1].replace(')', '')))
        sample_sizes.append(len(research_data))
    
    if len(all_proportions) == 0:
        print("No valid data found for any research type!")
        return None, None
    
    # Convert to numpy array for easier plotting
    proportions_array = np.array(all_proportions).T  # Transpose so categories are rows, research types are columns
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Get colors
    crest_colors = sns.color_palette("crest", n_colors=len(startup_categories))
    
    # Create stacked bars
    bottom = np.zeros(len(research_names))
    
    for i, category in enumerate(startup_categories):
        if i < len(proportions_array):
            bars = ax.bar(research_names, proportions_array[i], 
                         bottom=bottom, label=category, color=crest_colors[i])
            bottom += proportions_array[i]
    
    # Customize the plot
    ax.set_ylabel('Proportion', fontsize=12)
    ax.set_xlabel('Research Type', fontsize=12)
    ax.set_title('Startup Package Distribution by Research Type', fontsize=12, )
    ax.set_ylim(0, 1)
    
    # Add sample sizes to x-axis labels
    x_labels_with_n = [f"{name}\n(n={n})" for name, n in zip(research_names, sample_sizes)]
    ax.set_xticklabels(x_labels_with_n)
    
    # Create legend (reversed to match stacking order)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Startup Package Range', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    return fig, ax

In [ ]:
# Create the combined plot for research type vs. startup package
fig, ax = create_combined_research_startup_plot(
    data=df_scientist_melted,
    startup_col='If yes, what is the approximate value of your startup package?'
)

if fig is not None:
    plt.show()

## Heatmap of Startup vs. Salary

In [ ]:
startup_order = [
    'Under $250,000',
    '$250,000-$499,999', 
    '$500,000-$1,000,000',
    'Over $1,000,000'
]

plt.figure(figsize=(6, 4))

# Filter out None values from startup package before creating crosstab
df_filtered = df_scientist_melted[
    df_scientist_melted['If yes, what is the approximate value of your startup package?'] != 'None'
]

# Create custom mako colormap that doesn't start with black
from matplotlib.colors import ListedColormap
mako_cmap = sns.color_palette("mako", as_cmap=True)
# Create a truncated version that starts from a lighter color (skip the darkest 20%)
mako_colors = mako_cmap(np.linspace(0.2, 1.0, 256))
custom_mako = ListedColormap(mako_colors)

# Create crosstab and reindex with proper startup package order
crosstab_data = pd.crosstab(
    df_filtered['If yes, what is the approximate value of your startup package?'],
    df_filtered['salary_range_offered']
).dropna()

# Reindex to ensure proper ordering of startup packages
crosstab_data = crosstab_data.reindex(startup_order, fill_value=0)

# Create the heatmap with prettier styling
sns.heatmap(
    crosstab_data,
    annot=True, 
    fmt='d', 
    cmap=custom_mako,  
    linecolor='white',
    linewidths=0.5,
    cbar_kws={'label': 'Number of Responses'},
    square=False,
    annot_kws={'fontsize': 10, 'fontweight': 'bold'}
)

# Customize the plot
plt.title('Startup Package vs Salary Range Distribution', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Salary Range', fontsize=12, fontweight='bold')
plt.ylabel('Startup Package Value', fontsize=12, fontweight='bold')

# Rotate x-axis labels for better readability
plt.xticks(rotation=35, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)

# Improve layout
plt.tight_layout()
plt.savefig('fig_ps_heatmap.pdf', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# CE Job Offer Melting and Analysis

In [ ]:
def melt_job_offers_ce(df):
    """
    Melt a wide dataframe with multiple job offers into long format
    where each row represents one job offer with demographics for Clinician Educators.
    """
    
    # Define demographic columns (constant across all job offers)
    demographic_cols = [
        "Do you agree to participate in this survey? By clicking 'I Agree', you are consenting to participate in this survey. ",
        'Degrees Held (Select all that apply) (choice=MD, DO, MBBS or equivalent)',
        'Degrees Held (Select all that apply) (choice=MS (including MSCE, MSCI, MSci, MPH, etc.))',
        'Degrees Held (Select all that apply) (choice=PhD)',
        'Degrees Held (Select all that apply) (choice=Other)',
        'If Other for Degrees Held, please specify:',
        'Did you accept a job at the same institution where you completed fellowship training?',
        'Gender',
        'If you prefer to self-describe your gender: ',
        'Race/Ethnicity (Select all that apply) (choice=Asian or Asian American)',
        'Race/Ethnicity (Select all that apply) (choice=Black or African American)',
        'Race/Ethnicity (Select all that apply) (choice=Hispanic, Latino, Latina, or LatinX)',
        'Race/Ethnicity (Select all that apply) (choice=White)',
        'Race/Ethnicity (Select all that apply) (choice=Other)',
        'Race/Ethnicity (Select all that apply) (choice=Prefer not to answer)',
        'Do you identify as underrepresented in medicine?',
        'Type of Job Most Recently Accepted',
         'multiple_interviews',
         'extra_degree',
         'gender'
    ]
    
    # Define job offer columns without free text responses
    job_offer_base_cols = [
        'What geographic region is the institution?',
        'Did you interview at and receive offers from multiple institutions?',
        'Cost of living at institution of job negotiation',
        'What salary range were you offered?',
        'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?',
        'In your most recently accepted offer, what is your percent clinical time?',
        'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))',
        'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))',
        'What is the non-clinical full-time equivalent for? (choice=CME role)',
        'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)',
        'What is the non-clinical full-time equivalent for? (choice=Clinical program building)',
        'What is the non-clinical full-time equivalent for? (choice=Quality improvement)',
        'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)',
        'What is the non-clinical full-time equivalent for? (choice=Other)',
        'Did you receive non-clinical FTE?',
        'Do you have additional FTE dedicated for research?',
        "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0",
        'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?',
        'Is there transparency about how salary equity is evaluated at this institution?',
        'Were any signing bonuses or performance-based incentives part of your negotiation?',
        'Value of bonuses/incentives',
        'Were you offered a relocation package?',
        'If yes, what was the total value of the relocation package?',
        'Do you have a noncompete clause?',
        'Why did you choose to accept this job over other offers?'
    ]
    
    # Create mappings for each job offer
    job_offer_mappings = {
        'job_offer_1': {
            'What geographic region is the institution?': 'What geographic region is the institution?.3',
            'Did you interview at and receive offers from multiple institutions?': 'Did you interview at and receive offers from multiple institutions?.1',
            'Cost of living at institution of job negotiation': 'Cost of living at institution of job negotiation',
            'What salary range were you offered?': 'What salary range were you offered?.3',
            'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?': 'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?',
            'In your most recently accepted offer, what is your percent clinical time?': 'In your most recently accepted offer, what is your percent clinical time?',
            'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))': 'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))',
            'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))': 'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))',
            'What is the non-clinical full-time equivalent for? (choice=CME role)': 'What is the non-clinical full-time equivalent for? (choice=CME role)',
            'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)': 'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)',
            'What is the non-clinical full-time equivalent for? (choice=Clinical program building)': 'What is the non-clinical full-time equivalent for? (choice=Clinical program building)',
            'What is the non-clinical full-time equivalent for? (choice=Quality improvement)': 'What is the non-clinical full-time equivalent for? (choice=Quality improvement)',
            'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)': 'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)',
            'What is the non-clinical full-time equivalent for? (choice=Other)': 'What is the non-clinical full-time equivalent for? (choice=Other)',
            'Did you receive non-clinical FTE?': 'Did you receive non-clinical FTE?',
            'Do you have additional FTE dedicated for research?': 'Do you have additional FTE dedicated for research?',
            "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0": "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0",
            'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?': 'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?',
            'Is there transparency about how salary equity is evaluated at this institution?': 'Is there transparency about how salary equity is evaluated at this institution?',
            'Were any signing bonuses or performance-based incentives part of your negotiation?': 'Were any signing bonuses or performance-based incentives part of your negotiation?.3',
            'Value of bonuses/incentives': 'Value of bonuses/incentives',
            'Were you offered a relocation package?': 'Were you offered a relocation package?.3',
            'If yes, what was the total value of the relocation package?': 'If yes, what was the total value of the relocation package?.3',
            'Do you have a noncompete clause?': 'Do you have a noncompete clause?.3',
            'Why did you choose to accept this job over other offers?': 'Why did you choose to accept this job over other offers?.1'
        },
        'job_offer_2': {
            'Cost of living at institution of job negotiation': 'Cost of living at institution of job negotiation.1',
            'What salary range were you offered?': 'What salary range were you offered?.4',
            'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?': 'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?.1',
            'In your most recently accepted offer, what is your percent clinical time?': 'In your most recently accepted offer, what is your percent clinical time?.1',
            'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))': 'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director)).1',
            'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))': 'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director)).1',
            'What is the non-clinical full-time equivalent for? (choice=CME role)': 'What is the non-clinical full-time equivalent for? (choice=CME role).1',
            'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)': 'What is the non-clinical full-time equivalent for? (choice=Other medical educator role).1',
            'What is the non-clinical full-time equivalent for? (choice=Clinical program building)': 'What is the non-clinical full-time equivalent for? (choice=Clinical program building).1',
            'What is the non-clinical full-time equivalent for? (choice=Quality improvement)': 'What is the non-clinical full-time equivalent for? (choice=Quality improvement).1',
            'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)': 'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties).1',
            'What is the non-clinical full-time equivalent for? (choice=Other)': 'What is the non-clinical full-time equivalent for? (choice=Other).1',
            'Did you receive non-clinical FTE?': 'Did you receive non-clinical FTE?.1',
            'Do you have additional FTE dedicated for research?': 'Do you have additional FTE dedicated for research?.1',
            "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0": "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0.1",
            'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?': 'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?.1',
            'Is there transparency about how salary equity is evaluated at this institution?': 'Is there transparency about how salary equity is evaluated at this institution?.1',
            'Were any signing bonuses or performance-based incentives part of your negotiation?': 'Were any signing bonuses or performance-based incentives part of your negotiation?.4',
            'Value of bonuses/incentives': 'Value of bonuses/incentives.1',
            'Were you offered a relocation package?': 'Were you offered a relocation package?.4',
            'If yes, what was the total value of the relocation package?': 'If yes, what was the total value of the relocation package?.4',
            'Do you have a noncompete clause?': 'Do you have a noncompete clause?.4',
            'Did you interview at and receive offers from yet another institution?': 'Did you interview at and receive offers from yet another institution?.1'
        },
        'job_offer_3': {
            'Cost of living at institution of job negotiation': 'Cost of living at institution of job negotiation.2',
            'What salary range were you offered?': 'What salary range were you offered?.5',
            'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?': 'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?.2',
            'In your most recently accepted offer, what is your percent clinical time?': 'In your most recently accepted offer, what is your percent clinical time?.2',
            'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))': 'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director)).2',
            'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))': 'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director)).2',
            'What is the non-clinical full-time equivalent for? (choice=CME role)': 'What is the non-clinical full-time equivalent for? (choice=CME role).2',
            'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)': 'What is the non-clinical full-time equivalent for? (choice=Other medical educator role).2',
            'What is the non-clinical full-time equivalent for? (choice=Clinical program building)': 'What is the non-clinical full-time equivalent for? (choice=Clinical program building).2',
            'What is the non-clinical full-time equivalent for? (choice=Quality improvement)': 'What is the non-clinical full-time equivalent for? (choice=Quality improvement).2',
            'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)': 'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties).2',
            'What is the non-clinical full-time equivalent for? (choice=Other)': 'What is the non-clinical full-time equivalent for? (choice=Other).2',
            'Did you receive non-clinical FTE?': 'Did you receive non-clinical FTE?.2',
            'Do you have additional FTE dedicated for research?': 'Do you have additional FTE dedicated for research?.2',
            "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0": "Does a designation of 'core faculty' within residency or fellowship training programs get allocated FTE at this institution?\xa0.2",
            'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?': 'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?.2',
            'Is there transparency about how salary equity is evaluated at this institution?': 'Is there transparency about how salary equity is evaluated at this institution?.2',
            'Were any signing bonuses or performance-based incentives part of your negotiation?': 'Were any signing bonuses or performance-based incentives part of your negotiation?.5',
            'Value of bonuses/incentives': 'Value of bonuses/incentives.2',
            'Were you offered a relocation package?': 'Were you offered a relocation package?.5',
            'If yes, what was the total value of the relocation package?': 'If yes, what was the total value of the relocation package?.5',
            'Do you have a noncompete clause?': 'Do you have a noncompete clause?.5',
            'Did you interview at and receive offers from yet another institution? (You will not be asked to complete another set of questions.)': 'Did you interview at and receive offers from yet another institution? (You will not be asked to complete another set of questions.).1'
        }
    }
    
    # Create list to store melted dataframes
    melted_dfs = []
    
    # Create respondent ID if not exists
    if 'respondent_id' not in df.columns:
        df = df.reset_index()
        df['respondent_id'] = df.index
    
    # Filter demographic columns to only include those that exist in the dataframe
    existing_demographic_cols = [col for col in demographic_cols if col in df.columns]
    
    # Process each job offer
    for job_offer_name, column_mapping in job_offer_mappings.items():
        # Get columns that actually exist in the dataframe
        existing_columns = {k: v for k, v in column_mapping.items() if v in df.columns}
        
        if not existing_columns:
            continue
            
        # Create a dataframe for this job offer
        job_df = df[existing_demographic_cols + ['respondent_id']].copy()
        
        # Add job offer identifier
        job_df['job_offer_number'] = job_offer_name
        
        # Map the job offer columns
        for standard_col, actual_col in existing_columns.items():
            job_df[standard_col] = df[actual_col]
        
        # Remove rows where all job offer columns are null
        job_offer_cols = list(existing_columns.keys())
        job_df = job_df.dropna(subset=job_offer_cols, how='all')
        
        melted_dfs.append(job_df)
    
    # Combine all job offers
    if melted_dfs:
        final_df = pd.concat(melted_dfs, ignore_index=True)
        
        # Reorder columns for better readability
        cols_order = ['respondent_id', 'job_offer_number'] + existing_demographic_cols + job_offer_base_cols
        # Only include columns that exist in the dataframe
        cols_order = [col for col in cols_order if col in final_df.columns]
        final_df = final_df[cols_order]
        
        return final_df
    else:
        return pd.DataFrame()

# Example usage:
df_educator_melted = melt_job_offers_ce(df_educator)
print(f"Original shape: {df_educator.shape}")
print(f"Melted shape: {df_educator_melted.shape}")
print(f"Number of unique respondents: {df_educator_melted['respondent_id'].nunique()}")
print(f"Job offers per respondent: {df_educator_melted.groupby('respondent_id')['job_offer_number'].count().describe()}")

In [ ]:
df_educator_melted['internal_offer'] = ((df_educator_melted['Did you accept a job at the same institution where you completed fellowship training?'] == 'Yes') & 
                              (df_educator_melted['job_offer_number'] == 'job_offer_1')).apply(lambda x: True if x else False)

In [ ]:
df_educator_melted['internal_offer'].value_counts()

In [ ]:
df_educator_melted.to_csv('df_educator_melted.csv', index=False)

In [ ]:
def drop_empty_cost_of_living(df):
    col = 'Cost of living at institution of job negotiation'
    # Remove rows where value is None, NaN, or empty string
    return df[df[col].notnull() & (df[col] != '')]
df_educator_melted = drop_empty_cost_of_living(df_educator_melted)
df_educator_melted

In [ ]:
df_educator_melted.groupby('respondent_id')['job_offer_number'].count().value_counts()

## CE Job Offers TableOne

In [ ]:
columns = [ 'What geographic region is the institution?',
       'Cost of living at institution of job negotiation',
       'What salary range were you offered?',
       'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?',
       'In your most recently accepted offer, what is your percent clinical time?',
       'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))',
       'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))', 
      #  'What is the non-clinical full-time equivalent for? (choice=CME role)',
       'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)',
       'What is the non-clinical full-time equivalent for? (choice=Clinical program building)',
       'What is the non-clinical full-time equivalent for? (choice=Quality improvement)',
       'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)',
       'What is the non-clinical full-time equivalent for? (choice=Other)',
       'Did you receive non-clinical FTE?',
       'Do you have additional FTE dedicated for research?',
          'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?',
       'Is there transparency about how salary equity is evaluated at this institution?',
       'Were any signing bonuses or performance-based incentives part of your negotiation?',
       'Value of bonuses/incentives', 'Were you offered a relocation package?',
       'If yes, what was the total value of the relocation package?',
       'Do you have a noncompete clause?',]



# Generate TableOne
table1 = TableOne(df_educator_melted, columns=columns, categorical=columns,  rename=None, groupby='gender', pval=True)
table1.to_csv('table1_job_offers_educator_byGender.csv')
# table1

## CE Clinical Time Analysis

In [ ]:
ordered_categories = [
    "Less than 50%",
    "50%",
    "50-75%",
    "75-89%",
    "90-99%",
    "100%"
]

# Convert the column to a categorical type with the specified order
df_educator_melted['percent_clinical_ordered'] = pd.Categorical(
    df_educator_melted['In your most recently accepted offer, what is your percent clinical time?'],
    categories=ordered_categories,
    ordered=True
)

fig, ax = plt.subplots(figsize=(6, 3))
flare_colors = sns.color_palette("flare", n_colors=len(ordered_categories))
counts = df_educator_melted['percent_clinical_ordered'].value_counts()
counts = counts.reindex(ordered_categories)
counts.plot(kind='barh', ax=ax, color=flare_colors, edgecolor='black', linewidth=0.5)
plt.tight_layout()
plt.xlabel('Count', fontsize=12)
plt.ylabel('%FTE Clinical Time', fontsize=12)
plt.show()

In [ ]:
ordered_categories = [
    "Less than 50%",
    "50%",
    "50-75%",
    "75-89%",
    "90-99%",
    "100%"
]

# Convert the column to a categorical type with the specified order
df_educator_melted['percent_clinical_ordered'] = pd.Categorical(
    df_educator_melted['In your most recently accepted offer, what is your percent clinical time?'],
    categories=ordered_categories,
    ordered=True
)

# Create ordinal mapping for statistical testing
clinical_time_map = {cat: i+1 for i, cat in enumerate(ordered_categories)}
df_educator_melted['percent_clinical_ordinal'] = df_educator_melted['percent_clinical_ordered'].map(clinical_time_map)

# Binarize for 90% clinical time or more
df_educator_melted['clin_time_90plus'] = df_educator_melted['In your most recently accepted offer, what is your percent clinical time?'].isin(['90-99%', '100%'])

In [ ]:
def create_stacked_proportion_plot(df, category_col, ordinal_col, group_col, 
                                 ordered_categories, group_pairs=None, 
                                 title=None, xlabel=None, ylabel='Proportion',
                                 figsize=(6, 4), statistical_test='Mann-Whitney',
                                 print_stats=True):
       
    # Prepare data for stacked bar chart
    df_plot = df[[category_col, ordinal_col, group_col]].dropna()
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[group_col], df_plot[category_col])
    
    # Reorder columns based on category order
    crosstab = crosstab.reindex(columns=ordered_categories, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    flare_colors = sns.color_palette("flare", n_colors=len(ordered_categories))

    # Create the stacked bar plot with proportions
    fig, ax = plt.subplots(figsize=figsize)
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=flare_colors,)
    
    # Customize the plot
    ax.set_xlabel(xlabel or group_col)
    ax.set_ylabel(ylabel)
    ax.set_title(title or f'Distribution of {category_col} by {group_col}')
    ax.set_ylim(0, 1)  # Set y-axis from 0 to 1 for proportions
    
    # Fix legend order to match stacking order
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title=category_col.replace('_', ' ').title(), 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.tick_params(axis='x', rotation=45)
    
    # Add statistical annotation if pairs are provided
    if group_pairs:
        annotator = Annotator(ax, group_pairs, data=df_plot, x=group_col, y=ordinal_col)
        annotator.configure(test=statistical_test, loc='inside', text_format='simple')
        annotator.apply_and_annotate()
    
    plt.tight_layout()
    return fig, ax

In [ ]:
fig, ax = create_stacked_proportion_plot(
    df=df_educator_melted,
    category_col='percent_clinical_ordered',
    ordinal_col='percent_clinical_ordinal', 
    group_col='internal_offer',
    ordered_categories=ordered_categories,
    group_pairs=[(True,False)],
    title='Distribution of Clinical Time Percentage by Internal Offer',
    xlabel='Internal Offer'
)
plt.show()

In [ ]:
fig, ax = create_stacked_proportion_plot(
    df=df_educator_melted,
    category_col='percent_clinical_ordered',
    ordinal_col='percent_clinical_ordinal', 
    group_col='gender',
    ordered_categories=ordered_categories,
    group_pairs=[('Man','Woman')],
    title='Distribution of Clinical Time Percentage by Gender',
    xlabel='Gender'
)
plt.show()

In [ ]:
fig, ax = create_stacked_proportion_plot(
    df=df_educator_melted,
    category_col='percent_clinical_ordered',
    ordinal_col='percent_clinical_ordinal', 
    group_col='multiple_interviews',
    ordered_categories=ordered_categories,
    group_pairs=[(True,False)],
    title='Distribution of Clinical Time Percentage by Multiple Interviews',
    xlabel='Multiple Interviews'
)
plt.show()

In [ ]:
fig, ax = create_stacked_proportion_plot(
    df=df_educator_melted,
    category_col='percent_clinical_ordered',
    ordinal_col='percent_clinical_ordinal', 
    group_col='extra_degree',
    ordered_categories=ordered_categories,
    group_pairs=[(True,False)],
    title='Distribution of Clinical Time Percentage by Extra Degree',
    xlabel='Extra Degree'
)
plt.show()

In [ ]:
contingency_table = pd.crosstab(df_educator_melted['gender'],
                                 df_educator_melted['clin_time_90plus'])

fisher_exact(contingency_table)

In [ ]:
contingency_table = pd.crosstab(df_educator_melted['internal_offer'],
                                 df_educator_melted['clin_time_90plus'])

fisher_exact(contingency_table)

In [ ]:
contingency_table = pd.crosstab(df_educator_melted['extra_degree'],
                                 df_educator_melted['clin_time_90plus'])

fisher_exact(contingency_table)

In [ ]:
contingency_table = pd.crosstab(df_educator_melted['multiple_interviews'],
                                 df_educator_melted['clin_time_90plus'])

fisher_exact(contingency_table)

In [ ]:
ordered_categories = [
    "Less than 50%",
    "50%",
    "50-75%",
    "75-89%",
    "90-99%",
    "100%"
]

fig, ax = plt.subplots(figsize=(6, 3))
flare_colors = sns.color_palette("flare", n_colors=len(ordered_categories))
counts = df_educator_melted['percent_clinical_ordered'].value_counts()
counts = counts.reindex(ordered_categories)
counts.plot(kind='barh', ax=ax, color=flare_colors, edgecolor='black', linewidth=0.5)
plt.tight_layout()
plt.xlabel('Count', fontsize=12)
plt.ylabel('%FTE Clinical Time', fontsize=12)
plt.show()

In [ ]:
# Create binary categories for 90+ clinical time
binary_categories = ["<90% Clinical", "≥90% Clinical"]

# Binarize for 90% clinical time or more
df_educator_melted['clin_time_90plus'] = df_educator_melted['In your most recently accepted offer, what is your percent clinical time?'].isin(['90-99%', '100%'])

# Create a categorical column for the binary classification
df_educator_melted['clinical_time_binary'] = df_educator_melted['clin_time_90plus'].map({
    False: "<90% Clinical",
    True: "≥90% Clinical"
})

# Convert to categorical type with specified order
df_educator_melted['clinical_time_binary_ordered'] = pd.Categorical(
    df_educator_melted['clinical_time_binary'],
    categories=binary_categories,
    ordered=True
)

def create_stacked_proportion_plot_binary(df, category_col, group_col, 
                                        ordered_categories, group_pairs=None, 
                                        title=None, xlabel=None, ylabel='Proportion',
                                        figsize=(6, 4), statistical_test='Fisher-exact',
                                        print_stats=True):
       
    # Prepare data for stacked bar chart
    df_plot = df[[category_col, group_col]].dropna()
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[group_col], df_plot[category_col])
    
    # Reorder columns based on category order
    crosstab = crosstab.reindex(columns=ordered_categories, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    # Use a simple 2-color palette for binary comparison
    colors = ["#793341", "#643F86"] 
    
    # Create the stacked bar plot with proportions
    fig, ax = plt.subplots(figsize=figsize)
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=colors)
    
    # Customize the plot
    ax.set_xlabel(xlabel or group_col)
    ax.set_ylabel(ylabel)
    ax.set_title(title or f'Distribution of {category_col} by {group_col}')
    ax.set_ylim(0, 1)  # Set y-axis from 0 to 1 for proportions
    
    # Fix legend order to match stacking order
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Clinical Time', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.tick_params(axis='x', rotation=0)
    
    # Print Fisher's exact test results if requested
    if print_stats and group_pairs:
        from scipy.stats import fisher_exact
        
        print("Fisher's Exact Test Results:")
        print("=" * 40)
        
        # Get the contingency table for Fisher's exact test
        contingency_table = pd.crosstab(df_plot[group_col], df_plot[category_col])
        print("Contingency Table:")
        print(contingency_table)
        print()
        
        # Perform Fisher's exact test
        oddsratio, p_value = fisher_exact(contingency_table)
        
        print(f"Odds Ratio: {oddsratio:.4f}")
        print(f"P-value: {p_value:.4f}")
        
        # Interpret significance
        if p_value < 0.001:
            significance = "*** (p < 0.001)"
        elif p_value < 0.01:
            significance = "** (p < 0.01)"
        elif p_value < 0.05:
            significance = "* (p < 0.05)"
        else:
            significance = "ns (not significant)"
            
        print(f"Significance: {significance}")
        print()
        
        # Calculate proportions for interpretation
        prop_table = pd.crosstab(df_plot[group_col], df_plot[category_col], normalize='index')
        print("Proportion Table (row percentages):")
        print(prop_table.round(3))
    
    plt.tight_layout()
    return fig, ax

# Create the plot with binary clinical time categories
fig, ax = create_stacked_proportion_plot_binary(
    df=df_educator_melted,
    category_col='clinical_time_binary_ordered',
    group_col='internal_offer',
    ordered_categories=binary_categories,
    group_pairs=[(True, False)],
    title='Distribution of Clinical Time (≥90% vs <90%)',
    xlabel='Internal Offer',
    print_stats=True
)
plt.show()

In [ ]:
# Create the plot with binary clinical time categories
fig, ax = create_stacked_proportion_plot_binary(
    df=df_educator_melted,
    category_col='clinical_time_binary_ordered',
    group_col='extra_degree',
    ordered_categories=binary_categories,
    group_pairs=[(True, False)],
    title='Distribution of Clinical Time (≥90% vs <90%)',
    xlabel='extra_degree',
    print_stats=True
)
plt.show()

In [ ]:
# Create the plot with binary clinical time categories
fig, ax = create_stacked_proportion_plot_binary(
    df=df_educator_melted,
    category_col='clinical_time_binary_ordered',
    group_col='gender',
    ordered_categories=binary_categories,
    group_pairs=[(True, False)],
    title='Distribution of Clinical Time (≥90% vs <90%)',
    xlabel='Gender',
    print_stats=True
)
plt.show()

In [ ]:
# Create binary categories for 90+ clinical time
binary_categories = ["<90% Clinical", "≥90% Clinical"]

# Binarize for 90% clinical time or more
df_educator_melted['clin_time_90plus'] = df_educator_melted['In your most recently accepted offer, what is your percent clinical time?'].isin(['90-99%', '100%'])

# Create a categorical column for the binary classification
df_educator_melted['clinical_time_binary'] = df_educator_melted['clin_time_90plus'].map({
    False: "<90% Clinical",
    True: "≥90% Clinical"
})

# Convert to categorical type with specified order
df_educator_melted['clinical_time_binary_ordered'] = pd.Categorical(
    df_educator_melted['clinical_time_binary'],
    categories=binary_categories,
    ordered=True
)

def create_stacked_proportion_plot_binary(df, category_col, group_col, 
                                        ordered_categories, group_pairs=None, 
                                        title=None, xlabel=None, ylabel='Proportion',
                                        figsize=(6, 4), statistical_test='Fisher-exact',
                                        print_stats=True):
       
    # Prepare data for stacked bar chart
    df_plot = df[[category_col, group_col]].dropna()
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[group_col], df_plot[category_col])
    
    # Reorder columns based on category order
    crosstab = crosstab.reindex(columns=ordered_categories, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    # Use a red/purple color theme for binary comparison
    colors = ["#793341", "#643F86"]
    
    # Create the stacked bar plot with proportions
    fig, ax = plt.subplots(figsize=figsize)
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=colors)
    
    # Customize the plot
    ax.set_xlabel(xlabel or group_col)
    ax.set_ylabel(ylabel)
    ax.set_title(title or f'Distribution of {category_col} by {group_col}')
    ax.set_ylim(0, 1)  # Set y-axis from 0 to 1 for proportions
    
    # Fix legend order to match stacking order
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Clinical Time', 
              bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.tick_params(axis='x', rotation=0)
    
    # Print Fisher's exact test results if requested
    if print_stats and group_pairs:
        from scipy.stats import fisher_exact
        
        print("Fisher's Exact Test Results:")
        print("=" * 40)
        
        # Get the contingency table for Fisher's exact test
        contingency_table = pd.crosstab(df_plot[group_col], df_plot[category_col])
        print("Contingency Table:")
        print(contingency_table)
        print()
        
        # Perform Fisher's exact test
        oddsratio, p_value = fisher_exact(contingency_table)
        
        print(f"Odds Ratio: {oddsratio:.4f}")
        print(f"P-value: {p_value:.4f}")
        
        # Interpret significance
        if p_value < 0.001:
            significance = f"*** (p < 0.001)"
        elif p_value < 0.01:
            significance = f"** (p = {p_value:.3f})"
        elif p_value < 0.05:
            significance = f"* (p = {p_value:.3f})"
        else:
            significance = f"ns (p = {p_value:.3f})"
            
        print(f"Significance: {significance}")
        print()
        
        # Calculate proportions for interpretation
        prop_table = pd.crosstab(df_plot[group_col], df_plot[category_col], normalize='index')
        print("Proportion Table (row percentages):")
        print(prop_table.round(3))
    
    plt.tight_layout()
    return fig, ax

# Create the plot with binary clinical time categories
fig, ax = create_stacked_proportion_plot_binary(
    df=df_educator_melted,
    category_col='clinical_time_binary_ordered',
    group_col='internal_offer',
    ordered_categories=binary_categories,
    group_pairs=[(True, False)],
    title='Distribution of Clinical Time (≥90% vs <90%) by Internal Offer',
    xlabel='Internal Offer',
    print_stats=True
)
plt.show()

## CE Non-Clinical FTE Roles

In [ ]:
# Define the columns
non_clinical_columns = [
    'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))',
    'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))', 
    'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)',
    'What is the non-clinical full-time equivalent for? (choice=Clinical program building)',
    'What is the non-clinical full-time equivalent for? (choice=Quality improvement)',
    'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)',
    'What is the non-clinical full-time equivalent for? (choice=Other)'
]

# Create shorter labels for better visualization
short_labels = [
    'GME Role\n(Program Director)',
    'UME Role\n(Clerkship Director)', 
    'Other Medical\nEducator Role',
    'Clinical Program\nBuilding',
    'Quality\nImprovement',
    'Clinical Admin\nDuties',
    'Other'
]

# Count 'Checked' responses for each column
checked_counts = []
for i, col in enumerate(non_clinical_columns):
    if col == 'CME Role': # This column is commented out in the TableOne, so explicitly handle it
        checked_count = 0
    else:
        checked_count = (df_educator_melted[col] == 'Checked').sum()
    checked_counts.append(checked_count)

# Create DataFrame for plotting
plot_data = pd.DataFrame({
    'Role': short_labels,
    'Checked_Count': checked_counts
})

# Sort by count (optional)
plot_data = plot_data.sort_values('Checked_Count', ascending=False)

# Horizontal bar plot for better label readability
fig, ax = plt.subplots(figsize=(6, 4))

bars = ax.barh(range(len(plot_data)), plot_data['Checked_Count'],
               color='#A0306E',
               edgecolor='black', linewidth=0.5)

ax.set_title('Non-Clinical FTE Roles')
ax.set_xlabel('Number of offers with these roles')
# Set y-axis labels
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels(plot_data['Role'])

plt.tight_layout()
plt.show()

## CE Job Offers TableOne

In [ ]:
columns = [ 'What geographic region is the institution?',
       'Cost of living at institution of job negotiation',
       'What salary range were you offered?',
       'Did your initial contract include non-clinical FTE (e.g. Quality, safety, education, clinical program building)?',
       'In your most recently accepted offer, what is your percent clinical time?',
       'What is the non-clinical full-time equivalent for? (choice=GME role (e.g., residency or fellowship program director))',
       'What is the non-clinical full-time equivalent for? (choice=UME role (e.g., med student clerkship director))', 
      #  'What is the non-clinical full-time equivalent for? (choice=CME role)',
       'What is the non-clinical full-time equivalent for? (choice=Other medical educator role)',
       'What is the non-clinical full-time equivalent for? (choice=Clinical program building)',
       'What is the non-clinical full-time equivalent for? (choice=Quality improvement)',
       'What is the non-clinical full-time equivalent for? (choice=Clinical administrative duties)',
       'What is the non-clinical full-time equivalent for? (choice=Other)',
       'Did you receive non-clinical FTE?',
       'Do you have additional FTE dedicated for research?',
          'Is your salary adjusted based on your clinical effort (e.g., number of patients or RVUs)?',
       'Is there transparency about how salary equity is evaluated at this institution?',
       'Were any signing bonuses or performance-based incentives part of your negotiation?',
       'Value of bonuses/incentives', 'Were you offered a relocation package?',
       'If yes, what was the total value of the relocation package?',
       'Do you have a noncompete clause?',
       'clinical_time_binary_ordered',
       ]



# Generate TableOne
table1 = TableOne(df_educator_melted, columns=columns, categorical=columns,  rename=None, groupby='clinical_time_binary_ordered', pval=True)
# table1.to_csv('table1_job_offers_educator_byGender.csv')
table1

## CE Salary Range and Clinical Time Heatmap

In [ ]:
# Define the ordered categories for salary (CE specific)
salary_categories = [
    'Less than $150,000',
    '$150,000-$199,999',
    '$200,000-$249,999',
    '$250,000-$299,999',
    '$300,000-$349,999',
    '$350,000-$399,999',
    'More than $400,000'
]

# Convert to categorical with ordering
df_educator_melted['salary_range_ordered'] = pd.Categorical(df_educator_melted['What salary range were you offered?'], categories=salary_categories, ordered=True)
df_educator_melted['salary_range_ordered'].value_counts().sort_index()

In [ ]:
# Create the heatmap for clinical time vs. salary range
plt.figure(figsize=(8, 5))

sns.heatmap(
    pd.crosstab(df_educator_melted['percent_clinical_ordered'],
               df_educator_melted['salary_range_ordered']).dropna(),
    annot=True, 
    fmt='d', 
    cmap='flare',  
    linecolor='white',
    linewidths=0.5,
    cbar_kws={'label': 'Number of Responses'},
    square=False,
    annot_kws={'fontsize': 10, }
)

# Customize the plot
plt.title('Distribution of Clinical Time vs Salary Range', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Salary Range', fontsize=12, fontweight='bold')
plt.ylabel('Percent Clinical Time', fontsize=12, fontweight='bold')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)

# Improve layout
plt.tight_layout()

# Show the plot
plt.show()

# Combined Salary Data Analysis (PS and CE)

In [ ]:
df_scientist_melted_salary = df_scientist_melted[[ 'gender',
 'Type of Job Most Recently Accepted','internal_offer', 'salary_range_offered',
                                                'extra_degree','multiple_interviews',
 'What geographic region is the institution?',
 'Cost of living at institution (example cities)',
 'Were any signing bonuses or performance-based incentives part of your negotiation?',
 'If yes, what is the approximate total value of these bonuses/incentives?',]]

In [ ]:
df_educator_melted_salary = df_educator_melted[['gender',
 'Type of Job Most Recently Accepted','internal_offer', 'salary_range_ordered',
                                                'extra_degree','multiple_interviews',
 'What geographic region is the institution?',
 'Cost of living at institution of job negotiation',
 'Were any signing bonuses or performance-based incentives part of your negotiation?',
 'Value of bonuses/incentives',]]

In [ ]:
# Harmonize the column names to match between dataframes

# Rename columns in df_educator_melted_salary to match df_scientist_melted_salary
df_educator_melted_salary = df_educator_melted_salary.rename(columns={
    'salary_range_ordered': 'salary_range_offered',
    'Cost of living at institution of job negotiation': 'Cost of living at institution (example cities)',
    'Value of bonuses/incentives': 'If yes, what is the approximate total value of these bonuses/incentives?'
})

# Concatenate with standardized names
df_combined_salary = pd.concat([df_scientist_melted_salary, df_educator_melted_salary], 
                              ignore_index=True)

print("Combined dataframe shape:", df_combined_salary.shape)
print("\nColumn names:")
print(df_combined_salary.columns.tolist())

# Define a mapping for standardized column names for cleaner, shorter names
column_mapping = {
    'Type of Job Most Recently Accepted': 'job_type',
    'internal_offer': 'internal_offer',
    'salary_range_offered': 'salary_range',  # scientist version
    'salary_range_ordered': 'salary_range',  # educator version  
    'extra_degree': 'extra_degree',
    'multiple_interviews': 'multiple_interviews',
    'What geographic region is the institution?': 'geographic_region',
    'Cost of living at institution (example cities)': 'cost_of_living',  # scientist version
    'Cost of living at institution of job negotiation': 'cost_of_living',  # educator version
    'Were any signing bonuses or performance-based incentives part of your negotiation?': 'has_bonuses',
    'If yes, what is the approximate total value of these bonuses/incentives?': 'bonus_value',  # scientist version
    'Value of bonuses/incentives': 'bonus_value'  # educator version
}

# Apply standardized names to both dataframes
df_scientist_standardized = df_scientist_melted_salary.rename(columns=column_mapping)
df_educator_standardized = df_educator_melted_salary.rename(columns=column_mapping)

# Concatenate with standardized names
df_combined_standardized = pd.concat([df_scientist_standardized, df_educator_standardized], 
                                   ignore_index=True)

print("\nStandardized combined dataframe shape:", df_combined_standardized.shape)
print("\nStandardized column names:")
print(df_combined_standardized.columns.tolist())

# Match PS granularity with CE granularity for salary ranges
df_combined_standardized.replace({
    '$300,000-$349,999': 'More than $300,000',
    '$350,000-$399,999': 'More than $300,000',
    'More than $400,000': 'More than $300,000'
}, inplace=True)

# Define the ordered categories for salary for the combined dataframe
salary_categories = [
    'Less than $150,000',
    '$150,000-$199,999',
    '$200,000-$249,999',
    '$250,000-$299,999',
    'More than $300,000'
]

# Convert to categorical with ordering
df_combined_standardized['salary_range'] = pd.Categorical(df_combined_standardized['salary_range'], categories=salary_categories, ordered=True)
salary_map = {
    'Less than $150,000': 1,
    '$150,000-$199,999': 2,
    '$200,000-$249,999': 3,
    '$250,000-$299,999': 4,
    'More than $300,000': 5
}

df_combined_standardized['salary_ordinal'] = df_combined_standardized['salary_range'].map(salary_map)

In [ ]:
df_combined_standardized.to_csv('df_combined_salary_data.csv', index=False)

## Combined Salary Plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statannotations.Annotator import Annotator

# Define pastel rainbow colors (assuming these are defined in your environment)
# If not defined, uncomment the line below:
# pastel_rainbow = ['#FFB3BA', '#FFDFBA', '#FFFFBA', '#BAFFC9', '#BAE1FF', '#D4BAFF']

def create_stacked_bar_subplot(ax, data, x_col, y_col, order=None, pairs=None,
                               test='Mann-Whitney', loc='inside', text_format='simple',
                               title=None, xlabel=None, subplot_label=None):
    """
    Create a stacked bar chart in a subplot with statistical annotations
    """
    # Create salary mapping (consistent for combined data)
    salary_map = {
        'Less than $150,000': 1,
        '$150,000-$199,999': 2,
        '$200,000-$249,999': 3,
        '$250,000-$299,999': 4,
        'More than $300,000': 5
    }
    
    # Create reverse mapping for labels
    reverse_salary_map = {v: k for k, v in salary_map.items()}
    
    # Prepare data for stacked bar chart
    df_plot = data.copy()
    df_plot['salary_range_text'] = df_plot[y_col].map(reverse_salary_map)
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[x_col], df_plot['salary_range_text'])
    
    # Reorder columns based on salary_ordinal order
    salary_order = [reverse_salary_map[i] for i in sorted(reverse_salary_map.keys())]
    crosstab = crosstab.reindex(columns=salary_order, fill_value=0)
    
    # Reorder rows if order is specified
    if order:
        crosstab = crosstab.reindex(order, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    # Create the stacked bar plot
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=pastel_rainbow, legend=False)
    
    # Add subplot label (A, B, C, D)
    if subplot_label:
        ax.text(-0.15, 1.05, subplot_label, transform=ax.transAxes,
                fontsize=16, fontweight='bold', va='top')
    
    # Customize the plot
    ax.set_xlabel(xlabel or x_col, fontsize=12)
    ax.set_ylabel('Proportion', fontsize=12)
    ax.set_title(title or f'Distribution by {x_col}', fontsize=13)
    ax.set_ylim(0, 1)
    
    ax.tick_params(axis='x', rotation=0, labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    
    # Add sample sizes (n) below x-tick labels
    sample_sizes = crosstab.sum(axis=1)
    
    # Get the actual category labels from the crosstab index
    categories = crosstab.index.tolist()
    
    # Create new labels with sample sizes
    new_labels = []
    for i, cat in enumerate(categories):
        n_value = int(sample_sizes.iloc[i])
        # Format the label based on the type of data
        if isinstance(cat, bool):
            label_text = 'Yes' if cat else 'No'
        else:
            label_text = str(cat)
        new_labels.append(f'{label_text}\n(n={n_value})')
    
    ax.set_xticklabels(new_labels, fontsize=10, rotation=0)
    
    # Add statistical annotation if pairs are provided
    if pairs:
        annotator = Annotator(ax, pairs, data=data, x=x_col, y=y_col, order=order)
        annotator.configure(test=test, loc=loc, text_format=text_format)
        annotator.apply_and_annotate()
    
    return ax

# Create the main figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Salary Distribution Comparisons', fontsize=16, fontweight='bold', y=1.0)

# Flatten axes for easier indexing
axes = axes.flatten()

# (A) Physician-scientists vs Clinician-educator
create_stacked_bar_subplot(
    axes[0],
    data=df_combined_standardized,
    x_col='job_type',
    y_col='salary_ordinal',
    order=['Physician scientist', 'Clinician educator'],
    pairs=[('Physician scientist', 'Clinician educator')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Physician Scientist vs Clinician Educator',
    xlabel='Job Type',
    subplot_label='A)'
)

# (B) Internal vs External job offer
ax1 = create_stacked_bar_subplot(
    axes[1],
    data=df_combined_standardized,
    x_col='internal_offer',
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Internal vs External Job Offer',
    xlabel='Internal Offer',
    subplot_label='B)'
)

# Fix labels for Boolean values in subplot B
current_labels = [label.get_text() for label in ax1.get_xticklabels()]
new_labels_b = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_b.append(label.replace('Yes', 'Internal'))
    elif 'No' in label:
        new_labels_b.append(label.replace('No', 'External'))
    else:
        new_labels_b.append(label)
ax1.set_xticklabels(new_labels_b, fontsize=10, rotation=0)

# (C) Additional degree vs No additional degree
ax2 = create_stacked_bar_subplot(
    axes[2],
    data=df_combined_standardized,
    x_col='extra_degree',
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Additional Degree (MS, PhD, etc.)',
    xlabel='Has Additional Degree',
    subplot_label='C)'
)

# Fix labels for Boolean values in subplot C
current_labels = [label.get_text() for label in ax2.get_xticklabels()]
new_labels_c = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_c.append(label.replace('Yes', 'Additional Degree'))
    elif 'No' in label:
        new_labels_c.append(label.replace('No', 'No Additional Degree'))
    else:
        new_labels_c.append(label)
ax2.set_xticklabels(new_labels_c, fontsize=10, rotation=0)

# (D) Multiple offers vs Single offer
ax3 = create_stacked_bar_subplot(
    axes[3],
    data=df_combined_standardized,
    x_col='multiple_interviews',  # Assuming this should be 'multiple_offers' based on your description
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Multiple Offers vs Single Offer',
    xlabel='Received Multiple Offers',
    subplot_label='D)'
)

# Fix labels for Boolean values in subplot D
current_labels = [label.get_text() for label in ax3.get_xticklabels()]
new_labels_d = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_d.append(label.replace('Yes', 'Multiple Offers'))
    elif 'No' in label:
        new_labels_d.append(label.replace('No', 'Single Offer'))
    else:
        new_labels_d.append(label)
ax3.set_xticklabels(new_labels_d, fontsize=10, rotation=0)

# Create a single legend for all subplots
# Get legend handles and labels from the first subplot
handles, labels = axes[0].get_legend_handles_labels()
# Reverse order to match stacking order
fig.legend(reversed(handles), reversed(labels), 
           title='Salary Range',
           bbox_to_anchor=(0.95, .9), 
           loc='upper left',
           fontsize=11,
           title_fontsize=12)

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 0.92, 0.98])  # Leave space for legend on the right

plt.savefig('fig_salary_distribution_subplots.pdf', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Alternative: Save the figure

In [ ]:
salary_map = {
        'Less than $150,000': 1,
        '$150,000-$199,999': 2,
        '$200,000-$249,999': 3,
        '$250,000-$299,999': 4,
        'More than $300,000': 5
    }

df_combined_standardized.salary_ordinal.astype('float64').groupby(df_combined_standardized['job_type']).describe()

In [ ]:
# Refactored create_stacked_bar_with_stats for combined data
def create_stacked_bar_with_stats_combined(data, x_col, y_col, order=None, pairs=None, 
                                 test='Mann-Whitney', loc='outside', 
                                 text_format='simple', figsize=(7, 4), title=None, xlabel=None):
    
    # Create salary mapping (consistent for combined data)
    salary_map = {
        'Less than $150,000': 1,
        '$150,000-$199,999': 2,
        '$200,000-$249,999': 3,
        '$250,000-$299,999': 4,
        'More than $300,000': 5
    }
    
    # Create reverse mapping for labels
    reverse_salary_map = {v: k for k, v in salary_map.items()}
    
    # Prepare data for stacked bar chart
    df_plot = data.copy()
    df_plot['salary_range_text'] = df_plot[y_col].map(reverse_salary_map)
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[x_col], df_plot['salary_range_text'])
    
    # Reorder columns based on salary_ordinal order
    salary_order = [reverse_salary_map[i] for i in sorted(reverse_salary_map.keys())]
    crosstab = crosstab.reindex(columns=salary_order, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)

    # Create the stacked bar plot
    fig, ax = plt.subplots(figsize=figsize)
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=pastel_rainbow)
    
    # Customize the plot with larger font sizes
    ax.set_xlabel(xlabel or x_col, fontsize=14)
    ax.set_ylabel('Proportion', fontsize=14)
    ax.set_title(title or f'Distribution of Salary Ranges by {x_col}', fontsize=16)
    ax.set_ylim(0, 1)  # Set y-axis from 0 to 1 for proportions
    
    # Fix legend order to match stacking order (flip it)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Salary Range', 
              bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12, title_fontsize=13)
    
    ax.tick_params(axis='x', rotation=0, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    # Add sample sizes (n) below x-tick labels
    x_labels = ax.get_xticklabels()
    sample_sizes = crosstab.sum(axis=1)  # Get total counts for each category
    
    # Update x-tick labels to include sample sizes
    new_labels = []
    for i, label in enumerate(x_labels):
        original_text = label.get_text()
        n_value = sample_sizes.iloc[i] if i < len(sample_sizes) else 0
        new_labels.append(f'{original_text}\n(n={n_value})')
    
    ax.set_xticklabels(new_labels, fontsize=12)
    
    # Add statistical annotation if pairs are provided
    if pairs:
        annotator = Annotator(ax, pairs, data=data, x=x_col, y=y_col, order=order)
        annotator.configure(test=test, loc=loc, text_format=text_format)
        annotator.apply_and_annotate()
    
    plt.tight_layout()
    return fig, ax

# Plot for job type vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(
    data=df_combined_standardized, 
    x_col='job_type', 
    y_col='salary_ordinal', 
    order=['Physician scientist', 'Clinician educator'], 
    pairs=[('Physician scientist', 'Clinician educator')],
    test='Mann-Whitney',
    loc='inside', 
    text_format='simple', 
    figsize=(7, 4),
    title='Salary Offers by Job Type',
    xlabel='Job Type'
)
plt.show()

In [ ]:
# Plot for gender vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(
    data=df_combined_standardized, 
    x_col='gender', 
    y_col='salary_ordinal', 
    order=['Man', 'Woman'], 
    pairs=[('Man', 'Woman')],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges, All Jobs'
)
plt.show()

In [ ]:
# Plot for multiple interviews vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(
    df_combined_standardized,
    y_col='salary_ordinal',
    x_col='multiple_interviews',
    order=[True,False],
    pairs=[(True,False)],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges, All Jobs'
)
plt.show()

In [ ]:
# Plot for extra degree vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(
    df_combined_standardized,
    y_col='salary_ordinal',
    x_col='extra_degree',
    order=[True,False],
    pairs=[(True,False)],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges, All Jobs'
)
plt.show()

In [ ]:
# Plot for internal offer vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(
    df_combined_standardized,
    y_col='salary_ordinal',
    x_col='internal_offer',
    order=[True,False],
    pairs=[(True,False)],
    test='Mann-Whitney',loc='inside', 
    text_format='simple', figsize=(7, 4),
    title='Distribution of Salary Ranges, All Jobs'
)
plt.show()

In [ ]:
df_combined_standardized

In [ ]:
# Simplify geographic region and cost of living categories
geographic_region_map = {
    'Midwest (IL, IN, MI, OH, WI, IA, KS, MN, MO, NE, ND, SD)': 'Midwest',
    'Northeast (CT, ME, MA, NH, RI, VT, NJ, NY, PA)': 'Northeast', 
    'West (AZ, CO, ID, MT, NV, NM, UT, WY, AK, CA, HI, OR, WA)': 'West',
    'South (DE, DC, FL, GA, MD, NC, SC, VA, WV, AL, KY, MS, TN, AR, LA, OK, TX)': 'South'
}

df_combined_standardized['geographic_region_simple'] = df_combined_standardized['geographic_region'].map(geographic_region_map)

cost_living_map = {
    'Medium (e.g., Columbus, OH)': 'Medium',
    'High (e.g., New York, NY; San Francisco, CA)': 'High',
    'High (e.g., New York, NY; San Francisco, CA; Chicago, IL)': 'High',
    'Low (e.g., Wichita, KS)': 'Low'
}

df_combined_standardized['cost_of_living_simple'] = df_combined_standardized['cost_of_living'].map(cost_living_map)

# Convert to categorical with proper ordering (Low -> Medium -> High)
df_combined_standardized['cost_of_living_categorical'] = pd.Categorical(
    df_combined_standardized['cost_of_living_simple'], 
    categories=['Low', 'Medium', 'High'], 
    ordered=True
)

# Plot for geographic region vs. salary (combined data)
fig, ax = create_stacked_bar_with_stats_combined(df_combined_standardized, 'geographic_region_simple', 'salary_ordinal',
                                        title='Distribution of Salary Ranges by Geographic Region',
                                        xlabel='Geographic Region'
)
plt.show()

In [ ]:
import statsmodels.stats.multitest

fig, ax = plt.subplots(figsize=(6, 4))

data = df_combined_standardized
x = 'geographic_region_simple'
y = 'salary_ordinal'

# Remove any rows with missing values in the columns we're using
data_clean = data[[x, y]].dropna()

stats_results = []

# Perform pairwise Mann-Whitney U tests
for d1, d2 in itertools.combinations(data_clean[x].unique(), 2):
    days1 = data_clean[y][data_clean[x]==d1].dropna()
    days2 = data_clean[y][data_clean[x]==d2].dropna()
    if days1.size == 0 or days2.size == 0:
        continue
    pval = scipy.stats.mannwhitneyu(days1, days2).pvalue
    stats_results.append([d1, d2, days1.size, days2.size, pval])

# Convert to DataFrame
stats_results = pd.DataFrame(stats_results, columns=["group1", "group2",
                                                   "group1_size", "group2_size", "pval"])

# Apply FDR correction
stats_results["pval_adj"] = statsmodels.stats.multitest.fdrcorrection(stats_results.pval, alpha=0.05)[1]

# Get significant results
stat_results_sign = stats_results.loc[stats_results.pval_adj < 0.05, :]

# Create pairs for annotation
pairs = []
for _, r in stat_results_sign.iterrows():
    pairs.append((r.group1, r.group2))

# Create the box plot
sns.boxplot(data=data_clean, x=x, y=y, ax=ax, )

# Customize the plot
ax.set_xlabel('Geographic Region', fontsize=12)
ax.set_ylabel('Salary Range (Ordinal)', fontsize=12)
ax.set_title('Salary Distribution by Geographic Region', fontsize=14)

# Rotate x-axis labels for better readability
ax.tick_params(axis='x', labelsize=11)
trans = mpl.transforms.Affine2D().translate(6, 0)
for t in ax.get_xticklabels():
    t.set_rotation(15)
    t.set_horizontalalignment("right")
    t.set_transform(t.get_transform() + trans)

# Add statistical annotations if there are significant pairs
if len(pairs) > 0:
    annotator = Annotator(
        ax, 
        pairs, 
        data=data_clean, 
        x=x,
        y=y, 
        verbose=False
    )
    annotator._verbose = False
    annotator.configure(line_width=1)
    annotator.set_custom_annotations([f"p={x:.2e}" for x in stat_results_sign.pval_adj])
    annotator.annotate()
else:
    print("No significant pairwise differences found after FDR correction")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

data = df_combined_standardized
x = 'cost_of_living_simple'
y = 'salary_ordinal'

# Remove any rows with missing values in the columns we're using
data_clean = data[[x, y]].dropna()

stats_results = []

# Perform pairwise Mann-Whitney U tests
for d1, d2 in itertools.combinations(data_clean[x].unique(), 2):
    days1 = data_clean[y][data_clean[x]==d1].dropna()
    days2 = data_clean[y][data_clean[x]==d2].dropna()
    if days1.size == 0 or days2.size == 0:
        continue
    pval = scipy.stats.mannwhitneyu(days1, days2).pvalue
    stats_results.append([d1, d2, days1.size, days2.size, pval])

# Convert to DataFrame
stats_results = pd.DataFrame(stats_results, columns=["group1", "group2",
                                                   "group1_size", "group2_size", "pval"])

# Apply FDR correction
stats_results["pval_adj"] = statsmodels.stats.multitest.fdrcorrection(stats_results.pval, alpha=0.05)[1]

# Get significant results
stat_results_sign = stats_results.loc[stats_results.pval_adj < 0.05, :]

# Create pairs for annotation
pairs = []
for _, r in stat_results_sign.iterrows():
    pairs.append((r.group1, r.group2))

# Create the box plot
sns.boxplot(data=data_clean, x=x, y=y, ax=ax, )

# Customize the plot
ax.set_xlabel('Cost of Living', fontsize=12)
ax.set_ylabel('Salary Range (Ordinal)', fontsize=12)
ax.set_title('Salary Distribution by Cost of Living', fontsize=14)

# Rotate x-axis labels for better readability
ax.tick_params(axis='x', labelsize=11)
trans = mpl.transforms.Affine2D().translate(6, 0)
for t in ax.get_xticklabels():
    t.set_rotation(15)
    t.set_horizontalalignment("right")
    t.set_transform(t.get_transform() + trans)

# Add statistical annotations if there are significant pairs
if len(pairs) > 0:
    annotator = Annotator(
        ax, 
        pairs, 
        data=data_clean, 
        x=x,
        y=y, 
        verbose=False
    )
    annotator._verbose = False
    annotator.configure(line_width=1)
    annotator.set_custom_annotations([f"p={x:.3f}" for x in stat_results_sign.pval_adj])
    annotator.annotate()
else:
    print("No significant pairwise differences found after FDR correction")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np

# Set up variables
x = "cost_of_living_simple"
y = "salary_ordinal"
order = ['Low', 'Medium', 'High']

# Create reverse mapping for labels
reverse_salary_map = {v: k for k, v in salary_map.items()}

# Prepare data for stacked bar chart
df_plot = df_combined_standardized.copy()
df_plot['salary_range'] = df_plot[y].map(reverse_salary_map)
df_plot['salary_numeric'] = df_plot[y].map(salary_map)

# Create crosstab for stacked bar chart
crosstab = pd.crosstab(df_plot[x], df_plot['salary_range'])

# Convert to proportions
crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)

# Reorder columns based on salary_ordinal order
salary_order = [reverse_salary_map[i] for i in sorted(reverse_salary_map.keys())]
crosstab_prop = crosstab_prop.reindex(columns=salary_order, fill_value=0)

# Reorder rows (x-axis) based on order parameter
if order is not None:
    crosstab_prop = crosstab_prop.reindex(index=order, fill_value=0)

# Create the stacked bar plot
fig, ax = plt.subplots(figsize=(7, 4))
crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=pastel_rainbow)

# Customize the plot
ax.set_xlabel(x)
ax.set_ylabel('Proportion')
ax.set_title(f'Distribution of Salary Ranges by {x}')
ax.legend(title='Salary Range', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.tick_params(axis='x', rotation=0)

annot = Annotator(ax, [("Low", "Medium"), ("Low", "High")], data=df_combined_standardized, x=x, y=y, order=order)
annot.configure(test='Mann-Whitney', text_format='simple', loc='inside', verbose=2)
annot.apply_test()
ax, test_results = annot.annotate()

In [ ]:
df_combined_standardized['bonus_value'].value_counts()

In [ ]:
df_combined_standardized['has_bonuses'].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statannotations.Annotator import Annotator


pastel_rainbow = [
    "#F0C2C2",  # soft coral pink
    "#F0D2B8",  # soft peach
    "#FFFFBA",  # soft butter yellow
    "#C2F0C8",  # soft mint green
    "#C2D8F0",  # soft sky blue
    "#D8C2F0"   # soft lilac
]

def create_stacked_bar_subplot(ax, data, x_col, y_col, order=None, pairs=None,
                               test='Mann-Whitney', loc='inside', text_format='simple',
                               title=None, xlabel=None, subplot_label=None):
    """
    Create a stacked bar chart in a subplot with statistical annotations
    """
    # Create salary mapping (consistent for combined data)
    salary_map = {
        'Less than $150,000': 1,
        '$150,000-$199,999': 2,
        '$200,000-$249,999': 3,
        '$250,000-$299,999': 4,
        'More than $300,000': 5
    }
    
    # Create reverse mapping for labels
    reverse_salary_map = {v: k for k, v in salary_map.items()}
    
    # Prepare data for stacked bar chart
    df_plot = data.copy()
    df_plot['salary_range_text'] = df_plot[y_col].map(reverse_salary_map)
    
    # Create crosstab for stacked bar chart
    crosstab = pd.crosstab(df_plot[x_col], df_plot['salary_range_text'])
    
    # Reorder columns based on salary_ordinal order
    salary_order = [reverse_salary_map[i] for i in sorted(reverse_salary_map.keys())]
    crosstab = crosstab.reindex(columns=salary_order, fill_value=0)
    
    # Reorder rows if order is specified
    if order:
        crosstab = crosstab.reindex(order, fill_value=0)
    
    # Convert to proportions
    crosstab_prop = crosstab.div(crosstab.sum(axis=1), axis=0)
    
    # Create the stacked bar plot
    crosstab_prop.plot(kind='bar', stacked=True, ax=ax, color=pastel_rainbow, legend=False)
    
    # Add subplot label (A, B, C, D)
    if subplot_label:
        ax.text(-0.05, 1.09, subplot_label, transform=ax.transAxes,
                fontsize=16, fontweight='bold', va='top')
    
    # Customize the plot
    ax.set_xlabel(xlabel or x_col, fontsize=14)
    ax.set_ylabel('Proportion', fontsize=14)
    ax.set_title(title or f'Distribution by {x_col}', fontsize=13)
    ax.set_ylim(0, 1)
    
    ax.tick_params(axis='x', rotation=0, labelsize=10)
    ax.tick_params(axis='y', labelsize=14)
    
    # Add sample sizes (n) below x-tick labels
    sample_sizes = crosstab.sum(axis=1)
    
    # Get the actual category labels from the crosstab index
    categories = crosstab.index.tolist()
    
    # Create new labels with sample sizes
    new_labels = []
    for i, cat in enumerate(categories):
        n_value = int(sample_sizes.iloc[i])
        # Format the label based on the type of data
        if isinstance(cat, bool):
            label_text = 'Yes' if cat else 'No'
        else:
            label_text = str(cat)
        new_labels.append(f'{label_text}\n(n={n_value})')
    
    ax.set_xticklabels(new_labels, fontsize=16, rotation=0)
    
    # Add statistical annotation if pairs are provided
    if pairs:
        annotator = Annotator(ax, pairs, data=data, x=x_col, y=y_col, order=order)
        annotator.configure(test=test, loc=loc, text_format=text_format, 
                           comparisons_correction='bonferroni')
        annotator.apply_and_annotate()
    
    return ax

# Create the main figure with 3x2 subplots (5 panels total)
fig, axes = plt.subplots(3, 2, figsize=(15, 16))
fig.suptitle('Salary Distribution Comparisons', fontsize=16, fontweight='bold', y=1.00)

# Flatten axes for easier indexing
axes = axes.flatten()

# (A) Physician-scientists vs Clinician-educator
create_stacked_bar_subplot(
    axes[0],
    data=df_combined_standardized,
    x_col='job_type',
    y_col='salary_ordinal',
    order=['Physician scientist', 'Clinician educator'],
    pairs=[('Physician scientist', 'Clinician educator')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Physician Scientist vs Clinician Educator',
    xlabel='Job Type',
    subplot_label='A)'
)

# (B) Internal vs External job offer
ax1 = create_stacked_bar_subplot(
    axes[1],
    data=df_combined_standardized,
    x_col='internal_offer',
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Internal vs External Job Offer',
    xlabel='Internal Offer',
    subplot_label='B)'
)

# Fix labels for Boolean values in subplot B
current_labels = [label.get_text() for label in ax1.get_xticklabels()]
new_labels_b = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_b.append(label.replace('Yes', 'Internal'))
    elif 'No' in label:
        new_labels_b.append(label.replace('No', 'External'))
    else:
        new_labels_b.append(label)
ax1.set_xticklabels(new_labels_b, fontsize=10, rotation=0)

# (C) Additional degree vs No additional degree
ax2 = create_stacked_bar_subplot(
    axes[2],
    data=df_combined_standardized,
    x_col='extra_degree',
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Additional Degree (MS, PhD, etc.)',
    xlabel='Has Additional Degree',
    subplot_label='C)'
)

# Fix labels for Boolean values in subplot C
current_labels = [label.get_text() for label in ax2.get_xticklabels()]
new_labels_c = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_c.append(label.replace('Yes', 'Additional Degree'))
    elif 'No' in label:
        new_labels_c.append(label.replace('No', 'No Additional Degree'))
    else:
        new_labels_c.append(label)
ax2.set_xticklabels(new_labels_c, fontsize=14, rotation=0)

# (D) Multiple offers vs Single offer
ax3 = create_stacked_bar_subplot(
    axes[3],
    data=df_combined_standardized,
    x_col='multiple_interviews',  # Assuming this should be 'multiple_offers' based on your description
    y_col='salary_ordinal',
    order=[True, False],
    pairs=[(True, False)],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Multiple Offers vs Single Offer',
    xlabel='Received Multiple Offers',
    subplot_label='D)'
)

# Fix labels for Boolean values in subplot D
current_labels = [label.get_text() for label in ax3.get_xticklabels()]
new_labels_d = []
for label in current_labels:
    if 'Yes' in label:
        new_labels_d.append(label.replace('Yes', 'Multiple Offers'))
    elif 'No' in label:
        new_labels_d.append(label.replace('No', 'Single Offer'))
    else:
        new_labels_d.append(label)
ax3.set_xticklabels(new_labels_d, fontsize=14, rotation=0)

# (E) Cost of living by geographic region
ax4 = create_stacked_bar_subplot(
    axes[4],
    data=df_combined_standardized,
    x_col='cost_of_living_simple',
    y_col='salary_ordinal',
    order=['Low', 'Medium', 'High'],
    pairs=[('Low', 'Medium'), ('Low', 'High'), ('Medium', 'High')],
    test='Mann-Whitney',
    loc='inside',
    text_format='simple',
    title='Cost of Living by Geographic Region',
    xlabel='Cost of Living',
    subplot_label='E)'
)

# Hide the axes and spines for the empty subplot (bottom right - position 5 in 3x2 grid)
axes[5].axis('off')

# Create legend in the empty subplot space (row 3, column 2)
# Get legend handles and labels from the first subplot
handles, labels = axes[0].get_legend_handles_labels()

# Create legend in the empty axes[5] space
legend = axes[5].legend(reversed(handles), reversed(labels), 
                       title='Salary Range',
                       loc='center',
                       fontsize=14,
                       title_fontsize=16,
                       frameon=True,
                       fancybox=True,
                       shadow=False,
                       handlelength=2.5,
                       handletextpad=1.0,
                       columnspacing=2.0,
                       borderpad=1.5)

# Make legend title bold
legend.get_title().set_fontweight('bold')

# Adjust layout to optimize spacing
plt.tight_layout()

plt.savefig('fig_salary_distribution_subplots.pdf', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# TableOne: Combined Salary analysis

In [ ]:
columns = ['gender', 'job_type', 'internal_offer', 'salary_range', 'salary_ordinal','extra_degree',
       'multiple_interviews', 'geographic_region_simple','geographic_region', 
       'cost_of_living',
        'cost_of_living_simple',
       'cost_of_living_categorical',
       'has_bonuses', 'bonus_value', ]



# Generate TableOne
table1 = TableOne(df_combined_standardized, columns=columns, categorical=columns,  rename=None, groupby='job_type', pval=True, missing=False)
table1.to_csv('table1_job_offers_PS_CE.csv')
table1

# end